Cosumnes Model 
@author: Andrew

In [ ]:
# standard python utilities
import os
import sys
import glob
import pandas as pd
import numpy as np
import calendar
import time
from scipy.stats import gmean

# standard python plotting utilities
import matplotlib as mpl
import matplotlib.pyplot as plt

# standard geospatial python utilities
import pyproj # for converting proj4string
import shapely
import geopandas as gpd
import rasterio

# mapping utilities
import contextily as ctx
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm

In [ ]:
doc_dir = os.getcwd()
while os.path.basename(doc_dir) != 'Documents':
    doc_dir = os.path.dirname(doc_dir)
# dir of all gwfm data
gwfm_dir = os.path.dirname(doc_dir)+'/Box/research_cosumnes/GWFlowModel'
gwfm_dir

In [ ]:
flopy_dir = doc_dir+'/GitHub/flopy/'
if flopy_dir not in sys.path:
    sys.path.append(flopy_dir)
# sys.path
import flopy 

from importlib import reload
# importlib.reload
reload(flopy)

In [ ]:
# no_ss = True # remove steady state period
no_ss = False # keep steady state period

In [ ]:

# Transient -> might want to think about making SP1 steady
# calibration run dates
# end_date = pd.to_datetime('2021-09-30')
# strt_date = pd.to_datetime('2018-10-01')

# scenario run_dates dry -> wet -> dry
# end_date = pd.to_datetime('2020-09-30')
# strt_date = pd.to_datetime('2015-10-01')

end_date = pd.to_datetime('2020-09-30')
strt_date = pd.to_datetime('2016-10-01')


dates = pd.date_range(strt_date, end_date)

# The number of periods is the number of dates 
nper = len(dates) 
if no_ss == False:
    nper += 1 

# Each period has a length of one because the timestep is one day, have the 1st stress period be out of the date range
# need to have the transient packages start on the second stress period
perlen = np.ones(nper)
# steady state period can be 1 second
if no_ss == False:
    perlen[0] = 1/86400  # first period is steady state, rest are transient

# Steady or transient periods
steady = np.zeros(nper)
if no_ss == False:
    steady[0] = 1  # first period is steady state, rest are transient
steady = steady.astype('bool').tolist()
# Reduce the number of timesteps to decrease run time
# nstp = np.ones(nper)*np.append(np.ones(1),1*np.ones(nper-1))
nstp = np.ones(nper)

In [ ]:
# adjusters for boundary condition input
if no_ss == True:
    time_tr0 = 0  
    nper_tr = nper 
else:
    time_tr0 = 1
    nper_tr = nper-1


In [ ]:
#Maribeth's model parameters, had to switch nrow and ncol due to her issue in xul, yul
nrow=100
ncol=230
delr=200
delc=200
rotation=52.9

# The number of layers should be 1 for the Mehrten formation, 1 for the laguna plus the number of TPROGS layers,
# where the Laguna formation will be clipped by the TPROGS layers
# num_tprogs = 120 (max available below levelling), upscaling
max_num_layers =148 # based on thickness from -6m (1 m below DEM min) to -80m
upscale = 8
num_tprogs = int(max_num_layers/upscale)
num_leveling_layers = 1 # layers to create the upscaled unsaturated zone
nlay = 2 + num_leveling_layers + num_tprogs
# 120, is the number of tprogs layers that are useable below the -10 meter cutoff
tprog_thick = max_num_layers*0.5/num_tprogs


# There is essentially no difference bewtween WGS84 and NAD83 for UTM Zone 10N
# proj4_str='EPSG:26910'
proj4_str='+proj=utm +zone=10 +ellps=WGS84 +datum=WGS84 +units=m +no_defs '

In [ ]:
from flopy.utils.geometry import Polygon, LineString, Point
# Original model domain, 44.7 deg angle
# m_domain = gpd.read_file(gwfm_dir+'\\GWModelDomain_UTM10N\\GWModelDomain_Rec_UTM10N.shp')
# New model domain 52.9 deg
m_domain = gpd.read_file(gwfm_dir+'/DIS_data/NewModelDomain/GWModelDomain_52_9deg_UTM10N_WGS84.shp')

# Need to check this when changing model domains
xul, yul = list(m_domain.geometry.values[0].exterior.coords)[1]
list(m_domain.geometry.values[0].exterior.coords)
# m_domain.geometry.values[0].exterior

According to Flopy GitHub "Technically you need to create both a SpatialReference object and a ModelGrid object, but in practice the code looks very similar and can easily be implemented in one line."
WGS84 Zone 10N has EPSG: 32610  
Lower left corner of model is   
Zone 10 N  
Easting: 661211.18 m E  
Northing: 4249696.50 m N  
angle is approximate 53 degrees  

## Individual Users may change loadpath 
The default loadpath is set to an existing external hard drive for Andrew as F://
If the script doesn't find an external harddrive F:// then it will default to the C:// Drive in WRDAPP

In [ ]:
ext_dir = 'F:/WRDAPP'
c_dir = 'C:/WRDAPP'

if os.path.exists(ext_dir):
    loadpth = ext_dir 
elif os.path.exists(c_dir):
    loadpth = c_dir 

loadpth = loadpth +'/GWFlowModel/Cosumnes/levee_setback/streamflow/'
model_ws = loadpth+'historical_streamflow'

In [ ]:
# switch to modflow nwt to enable option bloack for use in owhm
m = flopy.modflow.Modflow(modelname = 'MF', exe_name = 'mf-owhm.exe', 
                          version = 'mfnwt', model_ws=model_ws)
# m = flopy.modflow.Modflow(modelname = 'MF', exe_name = 'mf2005', 
#                           version = 'mf2005', model_ws=model_ws)
#lenuni = 1 is in ft, lenuni = 2 is in meters
# itmuni is time unit 5 = years, 4=days, 3 =hours, 2=minutes, 1=seconds
dis = flopy.modflow.ModflowDis(nrow=nrow, ncol=ncol, 
                               nlay=nlay, delr=delr, delc=delc,
                               model=m, lenuni = 2, itmuni = 4,
                               xul = xul, yul = yul,rotation=rotation, proj4_str=proj4_str,
                              nper = nper, perlen=perlen, nstp=nstp, steady = steady,
                              start_datetime = strt_date)


In [ ]:
# xll, yll = list(m_domain.geometry.values[0].exterior.coords)[0]
# #Maribeth's model parameters, had to switch nrow and ncol due to her issue in xul, yul
# nrow=100
# ncol=230
# delr=np.repeat(200,ncol)
# delc=np.repeat(200,nrow)
# rotation=52.9
# modelgrid = flopy.discretization.StructuredGrid(xoff=xll, yoff=yll, proj4='EPSG:32610', angrot=rotation,
#                                    delr=delr, delc=delc, nrow=nrow,ncol=ncol)
# m.modelgrid.set_coord_info(xoff=xll, yoff=yll, proj4='EPSG:32610', angrot=angrot)
mg = m.modelgrid
# Write model grid to shapefile for later use
# mg.write_shapefile(gwfm_dir+'/DIS_data/grid/grid.shp', epsg = '32610')
# mg.write_shapefile(gwfm_dir+'/DIS_data/44_7_grid/44_7_grid.shp', epsg = '32610')


In [ ]:
# Load model grid as geopandas object
grid_p = gpd.read_file(gwfm_dir+'/DIS_data/grid/grid.shp')


# grid_p = gpd.read_file(gwfm_dir+'/DIS_data/44_7_grid/44_7_grid.shp')
# print(gwfm_dir)

# Find Michigan Bar location
# mb_gpd = sensors[sensors.Sensor_id == "MI_Bar"]
# mb_grid = gpd.sjoin(mb_gpd, grid_p, how = 'left', op = 'intersects')

In [ ]:
# Get vertexes of model domain
# ll = mg.get_coords(0, 0) #lower left
# lr = mg.get_coords(nrow*delr, 0) #lower right
# ur = mg.get_coords(nrow*delr, ncol*delc) #upper right
# ul = mg.get_coords(0, ncol*delc) #upper left
ll = mg.get_coords(0, 0) #lower left
lr = mg.get_coords(0, nrow*delr) #lower right
ur = mg.get_coords(ncol*delc, nrow*delr) #upper right
ul = mg.get_coords(ncol*delc, 0) #upper left
print(ll, lr, ur, ul)

# Shapefile of model bounds
from shapely.geometry import Polygon
vertices = np.stack(np.asarray((ll,lr, ur, ul)))
vertices
geoms = Polygon(vertices)

In [ ]:
# bi-linear raster sampling, aggregates cells to a size near the desired size then interpolates bewteen them 
# mean samplings, takes mean of all samples in the cell
# until 10/19/2022 the model had been using a bi-linear interpolation
# following this, it will be updated to a mean sampling scheme

In [ ]:
# np.savetxt(gwfm_dir+'\DIS_data\dem_44_7_200m_nearest.tsv', dem_data, delimiter = '\t')

# Based on Maribeth's grid aligned with Alisha's TPROGS model
# dem_data = np.loadtxt(gwfm_dir+'\DIS_data\dem_52_9_200m_nearest.tsv', delimiter = '\t')
# dem_data = np.loadtxt(gwfm_dir+'\DIS_data\dem_52_9_200m_linear.tsv', delimiter = '\t')
dem_data = np.loadtxt(gwfm_dir+'/DIS_data/dem_52_9_200m_mean.tsv')
# dem_data = np.loadtxt(gwfm_dir+'\DIS_data\dem_44_7_200m_linear_missing_right_corner.tsv', delimiter = '\t')

# dem_data = np.loadtxt(gwfm_dir+'\DIS_data\dem_44_7_200m_nearest.tsv', delimiter = '\t')

import seaborn as sns
sns.heatmap(dem_data, cmap = 'viridis', vmin = 0,square=True)

# Capture cross section of deeper geology

In [ ]:
# Pre-cretaceous metamorphic rocks - (variable thickness 200-500 ft thick)
# Ione formation (200 ft thick)
# Valley Springs formation (300 ft thick)
# Mehrten Formation (100 ft thick to 300 ft thick) (1-2 deg dip)
# Laguna Formation (less than 100 ft to between 200-300 ft thick) (less than 1 deg dip)
# upper formation (informed by well logs) (100 ft)
# ibound < 0 is constant head
# ibound = 0 is inactive cell
# ibound > 0 is active cell

## Need to adjust for change in model grid, based on Michigan Bar previously, maybe also look at effect of model domain angle vs cross section angle

In [ ]:

# The stream gage at michigan bar is now 13 columns in from the boundary
# mehrtenbound

# Cross section E appears to have an angle of 0 compared to the model domain,

In [ ]:
# # columns are xtop_miles, ytop_ft_amsl, xbot_miles, ytop_ft_amsl
# # XS upper bound should be at Michigan bar which is between Jackson road and Sacramento-Amador county line split
# # Mile 36 is approximately where Michigan bar aligns with the cross section
MB_XS_mile = 36
mehrtenbound = pd.read_csv(gwfm_dir+'/DIS_data/Mehrten_boundary_x_y.csv', parse_dates = False, 
                index_col = False, sep = ',', header = 'infer')
# Convert miles to feet and sets x value based on location of Michigan bar
# 0 is michigan bar and each major change in geologic dip is based on distance from Michigan Bar
mehrtenbound.xtop_miles = -5280*(MB_XS_mile - mehrtenbound.xtop_miles)
mehrtenbound.xbot_miles = -5280*(MB_XS_mile - mehrtenbound.xbot_miles)
# No flod boundary based on the original coordinates of the bottom of the Mehrten formation
mehrtenbound.noflow_x_miles = -5280*(MB_XS_mile - mehrtenbound.noflow_x_miles)

# East of mile 32 the entire vertical cross section, including up to the near entire surface
# is composed of old geologic formations that are not water bearing
volcanic_bound = (MB_XS_mile - 32)*-5280
# noflow_ind = int((1-(volcanic_bound/sumx))*ncol)

# Plot the x and y values
fig, ax = plt.subplots(1, 1)
mehrtenbound.plot(x = 'xtop_miles', y = 'ytop_ft_amsl', ax = ax)
mehrtenbound.plot(x = 'xbot_miles', y = 'ybot_ft_amsl', ax = ax)
plt.plot(-100*3.28*np.arange(0,len(dis.top[40,:])), np.flip(3.28*dis.top[40,:]))
# print(mehrtenbound)

In [ ]:

def xy_to_botm(xbound_ft, ybound_ft, nspace,ntransversespace):
    laybotm = np.zeros((ntransversespace, nspace))
    # Nspace will be either nrow or ncol depending model direction
    # ntransversespace is the opposite of nspace (ie nrow if nspace is ncol)
    # Calculate the distance between each major change in dip
    dx = np.diff(xbound_ft)
    # Scale by the total distance across the coordinates to get percentages
    sumx = np.sum(dx)
    dx /= sumx
    # Multiply the number of columns by the percent of columns in each section of constant dip
    dx *= nspace
    # Round the number of columns to allow proper use for indexing
    nx = np.round(dx).astype(int)
    # Fix any discrepancy in number of columns due to issues with rouding the percentages of columns
    # Add a column to the last set of columns because there is already uncertainty at the deeper end
    while(np.sum(nx)-nspace !=0):
        if np.sum(nx)-nspace <0:
            nx[-1] += 1
        elif np.sum(nx)-nspace >0:
            nx[-1] -= 1
    sum(nx)

    # Now split the coordinates into downsized coordinates in between each major change in dip
    k = 0
    for i in np.arange(0,len(nx)):
        for j in np.arange(0,ntransversespace):
            laybotm[j, k:k+nx[i]] = np.arange(ybound_ft[i],ybound_ft[i+1], -(ybound_ft[i]-ybound_ft[i+1])/nx[i])
        k += nx[i]
    return(laybotm)

In [ ]:
# X (east -west) and y (up-down vertical) of major dip changes for Mehrten Formation top boundary

mehrten_top = xy_to_botm(mehrtenbound.xtop_miles,mehrtenbound.ytop_ft_amsl, ncol, nrow)
# X (east -west) and y (up-down vertical) of major dip changes for Mehrten Formation bottom boundary
# drop na is because there are less values to mark changes in the bottom than top boundary
mehrten_bottom = xy_to_botm(mehrtenbound.xbot_miles.dropna(),mehrtenbound.ybot_ft_amsl.dropna(), ncol, nrow)

# Original x,y data for Mehrten bottom boundary to represent the noflow bounds
no_flow_bound = xy_to_botm(mehrtenbound.noflow_x_miles.dropna(), mehrtenbound.noflow_y_ft_amsl.dropna(),ncol,nrow)

In [ ]:
botm = np.zeros((nlay, nrow, ncol))
# Convert units from ft to meters and flip to match direction
botm[-2,:,:] = np.flip(mehrten_top/3.28)
botm[-1,:,:] = np.flip(mehrten_bottom/3.28)
no_flow_bound = np.flip(no_flow_bound/3.28)
dis.botm = botm
# dis.plot()

## Adjustment to bottom boundary to ensure sufficient top layer thickness for the TPROGS model
Although the bottom boundaries are being artifically lowered to allow for sufficient layer thickness, this will be corrected when ibound is implemented based on where the actual bottom boundary is and where there is high elevations based on likelihood to be volcanics geology.  
TPROGs extends from -80 to 80. The lowest point in the DEM is -5m, we should drop more than 1m below this to ensure adequate thickness for calculations. This leaves -6 m to -80 for TPROGs with standard upscaling, so 148 layers

In [ ]:
# The TPROGS model is 100m thick with some of it above the land surface
# to be safe, there should be at least 50 meters below ground surface to the bottom boundary

# Create 4 layers representing the upscaled "unsaturated" zone 
# ensures there is at least 1 m for each upscaled layer and rounded to create a clean boundary with the TPROGS data
leveling_layer_bottom = np.round(np.min(dem_data) - num_leveling_layers*1) - 1
# minimum thickness between top layer and Laguna?
leveling_layer_thickness = -10 # (dem_data - leveling_layer_bottom)/num_leveling_layers
botm[num_leveling_layers-1,:,:] = leveling_layer_bottom
for i in np.arange(num_leveling_layers-1,0,-1):
    botm[i-1,:,:] = botm[i,:,:] + leveling_layer_thickness
    
# Create TPROGS layers from top down
for i in np.arange(num_leveling_layers, num_leveling_layers + num_tprogs):
    botm[i,:,:] = botm[i-1,:,:] -tprog_thick
    
# Thickness to give to bottom layers below the TPROGS layers just to provide adequate spacing,
# this will be corrected by changing the geology in the layers above to account for what is actually in
# the Mehrten and what is in the Laguna formations, thickness of 5 also prevents any messy overlap
thickness_to_skip =10
# # Find where top boundary of Mehrten Formation rises within 10 meters of the top layer (10m for sufficient layer thickness)
bot3ind = np.min(np.where(botm[-2,:,:]>botm[-3,:,:]- thickness_to_skip)[1])

# # Where the top boundary of Mehrten was within 10 meters of the top layer 
# # set it equal to top layer elevation minus 10 for sufficient layer thickness
botm[-2,:,bot3ind:] = botm[-3,0,bot3ind]- thickness_to_skip
# # Repeat steps above for bottom of Mehrten formation with the top of the Mehrten formation
bot3ind = np.min(np.where(botm[-1,0,:]>botm[-2,0,:]- thickness_to_skip))
botm[-1,:,bot3ind:] = botm[-2,0,bot3ind]-thickness_to_skip

In [ ]:
# when you apply the 1/200 aspect the dips in the cell seem a lot less severe, so may just leave the layering as is
# for now with 4 layers representing the unsaturated zone
fig,ax = plt.subplots(figsize = (6,6))
# ax.set_aspect(aspect = 1/10)
plotrow = 80
plt.plot(dem_data[plotrow,:])

for i in np.arange(0,nlay):
    plt.plot(botm[i,plotrow,:])

In [ ]:
# Set the elevation of the top layer based on the DEM
m.dis.top = dem_data
# Bottom of model based on geology
m.dis.botm = botm
chk = dis.check()
chk.summary_array

## Complex ibound

### Define no flow cells based on elevation, informed by DWR cross sections and geologic maps of volcanic geology fingers leaving the mountains
In general, the location of Michigan Bar is near the boundary where there is total volcanics to majority alluvium. However there is a major finger North and South of the Cosumnes River of andesititc conglomerate, sandstone, breccia

In [ ]:
# Simplified ibound, only no flow cell if it is below the bottom of the Mehrten Formation
# Specify no flow boundary based on rough approx of geology (upper basin volcanics)
ibound = np.ones([nlay, nrow,ncol])


cutoff_elev = 56
ibound = ibound*(dem_data<cutoff_elev)

plt.imshow(ibound[0,:,:])

### Create a line bounding the noflow region to set the specified head boundary

In [ ]:
import pprint
from rasterio.features import shapes, rasterize

# The function shapes from rasterio requires uint8 format
ibound_line = ibound.astype(rasterio.uint8)
out = shapes(ibound_line,connectivity = 8)
alldata = list(out)

# maxl = 0
maxl = np.zeros(len(alldata))
for i in np.arange(0,len(alldata)):
    maxl[i] = len(alldata[i][0].get('coordinates')[0])
#     if len(alldata[i][0].get('coordinates')[0])>maxl:
#         maxl = len(alldata[i][0].get('coordinates')[0])
#         ind = i
# select the two longest linestring indexes (1st will be chunk down of divide (lower elevation) 2nd will chunk above (high elev))
maxl1, maxl2 = np.where(maxl>np.mean(maxl))[0]
print(maxl[maxl>np.mean(maxl)])

In [ ]:
temp = alldata[maxl2][0].get('coordinates')[0]
tl = LineString(temp)
tl

In [ ]:
from shapely.ops import LineString, linemerge, polygonize, unary_union
tl = LineString(temp)

# Get the constant head or general head boundary after the no flow cells
linerast = rasterio.features.rasterize([tl], out_shape = np.array((nrow,ncol)))
# remove far east bound line
linerast[:,ncol-1] = 0
fix_bound = np.min(np.argwhere(linerast[0,:]==1))
linerast[0,:] = 0
linerast[0,fix_bound]
np.shape(linerast)

# ibound[0,linerast==1] = -1

In [ ]:
from shapely.ops import LineString, linemerge, polygonize, unary_union
tl = LineString(temp)
tu = unary_union(tl)
poly = list(polygonize(tu))
# Set the polygon/raster for the top layer, no buffer needed
poly0 = poly[0].buffer(distance = 0)
polyrast0 = rasterio.features.rasterize([poly0], out_shape = np.array((nrow,ncol)))
# Set the polygon/raster for the top layer, slight buffer needed to expand geologic formation outward with depth as 
# naturally occurs
poly1 = poly[0].buffer(distance = 13)
polyrast1 = rasterio.features.rasterize([poly1], out_shape = np.array((nrow,ncol)))
# Set the polygon/raster for the bottom layer, largest buffer needed
poly2 = poly[0].buffer(distance = 17)
polyrast2 = rasterio.features.rasterize([poly2], out_shape = np.array((nrow,ncol)))

ibound = np.ones([nlay, nrow,ncol])
# Need to decide whether all layers or just the top layer are affected by ibound from elevation
# it is better to define the top layer with a simple dem>elevation check than the rasterize functins that isn't perfect
# ibound[0,polyrast0==1] = 0
# Need to decide whether all layers or just the top layer are affected by ibound from elevation
ibound[-2,polyrast1==1] = 0
# Need to decide whether all layers or just the top layer are affected by ibound from elevation
ibound[-1,polyrast2==1] = 0

# The bottom boundary has a dip of 1-2 degrees which is essentially a slope of 0.015 based on given cross section data
# The layer thickness for TPROGS
laythk = tprog_thick
# It appeared shapely buffer is on the scale of kilometers
run = (laythk/0.015)/1000
run_const = run
for i in np.arange(1,nlay-2):
    # error saying poly[i] is not subscriptable
    polyi = poly[0].buffer(distance = run)
    polyrast = rasterio.features.rasterize([polyi], out_shape = np.array((nrow,ncol)))
    # Need to decide whether all layers or just the top layer are affected by ibound from elevation
    ibound[i,polyrast==1] = 0
    run += run_const

In [ ]:
# wherever the constant head/specified head bound is the cells need to be active
ibound[0,dem_data>cutoff_elev] = 0
ibound[0,linerast==1] = 1
plt.imshow(ibound[0,:,:])
plt.colorbar()

In [ ]:
# copy the ibound array to alter the geology array to set these cells as low permeability formations
# either marine or volcanic based
deep_geology = np.invert(ibound[:,:,:].astype(bool))

# reset ibound to all active cells to reduce non-linearity
# still need to take account of no flow cells for lake package
ibound = np.ones([nlay, nrow,ncol])

In [ ]:
# np.savetxt(gwfm_dir+'/BAS6/deep_geology_'+str(nlay)'layer.tsv',np.reshape(deep_geology, (nlay*nrow,ncol)), delimiter ='\t')
# deep_geology = np.loadtxt(m.model_ws+'/input_data/deep_geology.tsv', delimiter ='\t')
# deep_geology = np.reshape(deep_geology, (nlay,nrow,ncol))

### Remove no flow cells in the first layer where there are stream cells

In [ ]:
# where the specified head boundary is the cells must be active
# ibound[0,chd_locs[0], chd_locs[1]] = -1

In [ ]:
# update steady state to be a best available head contours map
ghb_dir = gwfm_dir+'/GHB_data'

year = strt_date.year # 2016
filename = glob.glob(ghb_dir+'/final_WSEL_arrays/spring'+str(year)+'_kriged_WSEL.tsv')[0]
hd_strt = np.loadtxt(filename)
# where head is  above land surface set to 15 ft below land surface
hd_strt[hd_strt > m.dis.top.array] = m.dis.top.array[hd_strt > m.dis.top.array] - 15*0.3048

dtw_strt = m.dis.top.array - hd_strt
plt.imshow(hd_strt)
plt.colorbar(shrink=0.6)

In [ ]:
strt = np.ones((nlay, nrow, ncol), dtype = np.float32)
# The model should start in hydraulic connection
if no_ss == False:
    strt[:,:,:] = m.dis.top[:,:] #maybe the mean of starting heads i causing issues?

#start with adjusted head contours because the steady state is causing the model to start off
if no_ss ==True:
    strt[:,:,:] = hd_strt

In [ ]:
# bas_dir = gwfm_dir+'/BAS6/'
# head_ss_dir = 'historical_streamflow'
# # head_ss_dir = 'historical_SS' # hasn't been corrected for recharge yet
# for k in np.arange(0,m.dis.nlay):
#     strt[k,:,:] = np.loadtxt(bas_dir+'steadystate_heads/'+head_ss_dir+'/layer'+str(k)+'.tsv',delimiter='\t')

In [ ]:
# f = open(model_ws+'/MF.bas','r')


In [ ]:
# temp= f.readlines()
# temp[0].replace('FREE STOPERROR 0.01','FREE STOPERROR 0.01 NO_FAILED_CONVERGENCE_STOP')
# f.close()

In [ ]:
# not needed if switch to NWT solver

In [ ]:
# Basic package, BAS

# ibound < 0 is constant head
# ibound = 0 is inactive cell
# ibound > 0 is active cell
# strt is array of starting heads
# add option: STOPERROR 0.01 to reduce percent error when OWHM stops model
# if solver criteria are not met, the model will continue if model percent error is less than stoperror
bas = flopy.modflow.ModflowBas(model = m, ibound=ibound, strt = strt, stoper = 0.01)

In [ ]:
# may have to manually add since it seems to overwrite
# allows model to continue even if convergence fails
bas.options = bas.options +' NO_FAILED_CONVERGENCE_STOP'

In [ ]:
bas.check()
# bas.write_file()

## Northwest and Southeast GHB boundaries based on historical WSEL

In [ ]:
# raster cropping will be done in outside script so the only part read in will be the final array
ghb_dir = gwfm_dir+'/GHB_data'

In [ ]:
def load_ghb_wsel(strt_date, end_date):
    strtyear = strt_date.year
    endyear = end_date.year+1
    kriged_fall = np.zeros((int(endyear-strtyear),nrow,ncol))
    kriged_spring = np.zeros((int(endyear-strtyear),nrow,ncol))

    # keep track of which place in array matches to year
    year_to_int = np.zeros((endyear-strtyear,2))

#     t=0
    for t,year in enumerate(np.arange(strtyear,endyear)):
        # load and place spring kriged data in np array, load spring first
        filename = glob.glob(ghb_dir+'/final_WSEL_arrays/spring'+str(year)+'_kriged_WSEL.tsv')[0]
        df_grid = np.loadtxt(filename)
        kriged_spring[t,:,:] = df_grid

        # load and place fall kriged data in np array
        filename = glob.glob(ghb_dir+'/final_WSEL_arrays/fall'+str(year)+'_kriged_WSEL.tsv')[0]
        df_grid = np.loadtxt(filename)
        kriged_fall[t,:,:] = df_grid
        # track year and location
        year_to_int[t,0] = t
        year_to_int[t,1] = year
#         t+=1
    sy_ind = np.repeat(['spring','fall'],(endyear-strtyear)),np.tile(np.arange(strtyear,endyear),2)
    sy_ind = pd.MultiIndex.from_arrays(sy_ind, names=['season','year'])

    ## NW is row 0, SE is last row
    kriged_NW = np.vstack((kriged_spring[:,0,:],kriged_fall[:,0,:]))
    kriged_SE = np.vstack((kriged_spring[:,nrow-1,:],kriged_fall[:,nrow-1,:] ))
    
    y_num=0

    nwhead_fall = kriged_fall[:,0,:]
    sehead_fall = kriged_fall[:,-1,:]


    # Set kriged water table elevations that are above land surface to land surface
    nwhead_fall = np.where(nwhead_fall>dem_data[0,:], dem_data[0,:], nwhead_fall)
    sehead_fall = np.where(sehead_fall>dem_data[-1,:], dem_data[-1,:], sehead_fall)

    kriged_NW = np.where(kriged_NW>dem_data[0,:], dem_data[0,:], kriged_NW)
    kriged_SE = np.where(kriged_SE>dem_data[-1,:], dem_data[-1,:], kriged_SE)

    # calculate the average depth to water table for the spring and from 2013-2018 for the northwest and southeast boundary
    avg_nw = np.nanmean(kriged_NW,axis=0)
    avg_se = np.nanmean(kriged_SE,axis=0)

In [ ]:
strtyear = strt_date.year
endyear = end_date.year+1
kriged_fall = np.zeros((int(endyear-strtyear),nrow,ncol))
kriged_spring = np.zeros((int(endyear-strtyear),nrow,ncol))

kriged_NW = np.zeros((int(endyear-strtyear)*2,ncol))
kriged_SE = np.zeros((int(endyear-strtyear)*2,ncol))
# keep track of which place in array matches to year
year_to_int = np.zeros((endyear-strtyear,2))

t=0
for year in np.arange(strtyear,endyear):
    
    # load and place spring kriged data in np array, load spring first
    filename = glob.glob(ghb_dir+'/final_WSEL_arrays/spring'+str(year)+'_kriged_WSEL.tsv')[0]
    df_grid = np.loadtxt(filename)
    kriged_spring[t,:,:] = df_grid
    
    # load and place fall kriged data in np array
    filename = glob.glob(ghb_dir+'/final_WSEL_arrays/fall'+str(year)+'_kriged_WSEL.tsv')[0]
    df_grid = np.loadtxt(filename)
    kriged_fall[t,:,:] = df_grid

    
    # save NW  dataframe
#     kriged_NW[t] = kriged_spring[t,0,:]
#     kriged_NW[2*t] = kriged_fall[t,0,:]
#     # save SE data frame
#     kriged_SE[t] = kriged_spring[t,nrow-1,:]
#     kriged_SE[2*t] = kriged_fall[t,nrow-1,:]
    
    year_to_int[t,0] = t
    year_to_int[t,1] = year
    t+=1
    

In [ ]:
sy_ind = np.repeat(['spring','fall'],(endyear-strtyear)),np.tile(np.arange(strtyear,endyear),2)
sy_ind = pd.MultiIndex.from_arrays(sy_ind, names=['season','year'])
# sy_ind

In [ ]:
## NW is row 0, SE is last row
kriged_NW = np.vstack((kriged_spring[:,0,:],kriged_fall[:,0,:]))
kriged_SE = np.vstack((kriged_spring[:,nrow-1,:],kriged_fall[:,nrow-1,:] ))
fig,ax=plt.subplots(1,2, figsize=(12,4))
pd.DataFrame(np.rot90(kriged_NW),columns=sy_ind).loc[:,'spring'].plot(ax=ax[0],linestyle='--')
pd.DataFrame(np.rot90(kriged_NW),columns=sy_ind).loc[:,'fall'].plot(ax=ax[0])

pd.DataFrame(np.rot90(kriged_SE),columns=sy_ind).loc[:,'spring'].plot(ax=ax[1],linestyle='--')
pd.DataFrame(np.rot90(kriged_SE),columns=sy_ind).loc[:,'fall'].plot(ax=ax[1])

In [ ]:
# fig,ax=plt.subplots()
# for s in ['spring']:
#     for y in np.arange(2013,2018):
#         fn = ghb_dir+'/final_WSEL_arrays/'+s+str(y)+'_kriged_WSEL.tsv'
#         hd_arr = np.loadtxt(fn)
#         ax.plot(hd_arr[0,:],'--',label=y)
# for s in ['fall']:
#     for y in np.arange(2013,2018):
#         fn = ghb_dir+'/final_WSEL_arrays/'+s+str(y)+'_kriged_WSEL.tsv'
#         hd_arr = np.loadtxt(fn)
#         ax.plot(hd_arr[0,:],'-',label=y)
# plt.legend()
# # fall 2019 is too high, spring 2020 is too low

In [ ]:
y_num=0

nwhead_fall = kriged_fall[:,0,:]
sehead_fall = kriged_fall[:,-1,:]


# Set kriged water table elevations that are above land surface to land surface minus 15 ft (based on historical levels)
# nwhead_fall = np.where(nwhead_fall>dem_data[0,:], dem_data[0,:] - 15*0.3048, nwhead_fall)
# sehead_fall = np.where(sehead_fall>dem_data[-1,:], dem_data[-1,:]- 15*0.3048, sehead_fall)
dem_offset = 15*0.3048
# dem_offset = 0

kriged_NW = np.where(kriged_NW>dem_data[0,:], dem_data[0,:]- dem_offset, kriged_NW)
kriged_SE = np.where(kriged_SE>dem_data[-1,:], dem_data[-1,:]- dem_offset, kriged_SE)

# calculate the average depth to water table for the spring and from 2013-2018 for the northwest and southeast boundary
avg_nw = np.nanmean(kriged_NW,axis=0)
avg_se = np.nanmean(kriged_SE,axis=0)

In [ ]:
def get_layer_from_elev(elev, botm_slice, nlay):
    """  Return uppermost model layer occupied at least partly by some elevation data
    Parameters
    ----------
    elev: 1D array (n) with elevations matching model elevation units
    botm: 2D array (nlay, n) with layer elevations of model using same x,y locations at elev1D
    """
    elev_lay = np.zeros(len(elev))
    for k in np.arange(0,nlay-1):
        for j in np.arange(0,len(elev)):
            if botm_slice[k,j] > elev[j]:
                elev_lay[j] = k + 1
    return(elev_lay.astype(int))
                
nw_lay = get_layer_from_elev(avg_nw, botm[:,0,:], m.dis.nlay)
se_lay = get_layer_from_elev(avg_se, botm[:,-1,:], m.dis.nlay)

## Read in TPROGS data

In [ ]:
# "C:\Users\ajcalder\Box\research_cosumnes\Large_TPROGS_run\TPROGS_realizations\tsim_Cosumnes_Full_Model.asc1"
# create tprogs directory reference to 100 large tprogs runs ascii files
# tprogs_dir = os.path.dirname(gwfm_dir)+'/Large_TPROGS_run/Archive/TPROGS_realizations/'
# # get all file names
# tprogs_files = glob.glob(tprogs_dir+'*')


In [ ]:
mf_tprogs_dir = gwfm_dir+'/UPW_data/tprogs_final/'
tprogs_files = glob.glob(mf_tprogs_dir+'*')

gel_dir = gwfm_dir+'/UPW_data'
if 'ZonePropertiesInitial.csv' in os.listdir(model_ws):
    print('exists')
    params = pd.read_csv(model_ws+'/ZonePropertiesInitial.csv',index_col='Zone')
else:
    params = pd.read_csv(gel_dir+'/ZonePropertiesInitial.csv',index_col='Zone')
    params.to_csv(model_ws+'/ZonePropertiesInitial.csv')
# convert from m/s to m/d
params['K_m_d'] = params.K_m_s * 86400    

In [ ]:
tprogs_fxn_dir = doc_dir+'/GitHub/CosumnesRiverRecharge/tprogs_utilities'
if tprogs_fxn_dir not in sys.path:
    sys.path.append(tprogs_fxn_dir)
# sys.path
import tprogs_cleaning as tc

from importlib import reload
# importlib.reload
reload(tc)

In [ ]:
#  one based is  min:1, 90:mean, max:34
t=89 #0, 33,89
tprogs_info = [80, -80, 320]

tprogs_line = np.loadtxt(tprogs_files[t])
masked_tprogs= tc.tprogs_cut_elev(tprogs_line, dem_data, tprogs_info)
K, Sy, Ss= tc.int_to_param(masked_tprogs, params)

# save tprogs facies array as input data for use during calibration
tprogs_dim = masked_tprogs.shape
np.savetxt(model_ws+'/input_data/tprogs_facies_array.tsv', np.reshape(masked_tprogs, (tprogs_dim[0]*nrow,ncol)), delimiter='\t')
# masked_tprogs = np.reshape(np.loadtxt(model_ws+'/input_data/tprogs_facies_array.tsv', delimiter='\t'), (320,100,230))


In [ ]:
# t=89 #0, 33,89
# tprogs_line = np.loadtxt(tprogs_files[t])
# masked_tprogs= tc.tprogs_cut_elev(tprogs_line, 320*np.ones((nrow,ncol)))
# Kt, Syt, Sst= tc.int_to_param(masked_tprogs, params)

In [ ]:
# fig,ax=plt.subplots(figsize=(8,4))
# ax.imshow(Kt[0,:,:])

# plt.xlabel('Columns - 200 m')
# plt.ylabel('Rows - 200 m')
# plt.savefig(plt_dir+'top_view_tprogs_89.png',dpi=600,bbox_inches='tight')
# plt.show()

# fig,ax=plt.subplots(figsize=(8,4))
# ax.imshow(Kt[:,0,:],aspect=1/10)
# plt.xlabel('Columns - 200 m')
# plt.ylabel('Layers - 0.5 m')
# plt.title('1/10 Aspect Ratio')
# plt.savefig(plt_dir+'side_view_tprogs_89.png',dpi=600,bbox_inches='tight')


In [ ]:
# take the mean water surface elevations for fall and spring
kriged_fall_avg = np.nanmean(kriged_fall,axis=0)
kriged_spring_avg = np.nanmean(kriged_spring,axis=0)
# take average between fall and spring water surface elevations
kriged_fall_spring_avg = (kriged_fall_avg+kriged_spring_avg)/2

In [ ]:
# from scipy.stats import hmean
# unsat_hk = np.mean(tc.tprogs_cut_saturated(K,kriged_fall_spring_avg),axis=0)
# unsat_vka = hmean(tc.tprogs_cut_saturated(K,kriged_fall_spring_avg))
# unsat_sy = np.mean(tc.tprogs_cut_saturated(Sy,kriged_fall_spring_avg),axis=0)
# unsat_ss = np.mean(tc.tprogs_cut_saturated(Ss,kriged_fall_spring_avg),axis=0)

# # set any zero values to the average of the domain, should be added to the function above
# # arithmetic mean is used because we are filling data and not calculating vertical averages
# unsat_hk[unsat_hk.data==0] = np.mean(unsat_hk)
# unsat_vka[unsat_vka.data==0] = np.mean(unsat_vka)
# unsat_sy[unsat_sy.data==0] = np.mean(unsat_sy)
# unsat_ss[unsat_ss.data==0] = np.mean(unsat_ss)

In [ ]:
# the first layer starts at -10 m which corresponds to 20 layers below 0
# 0m AMSL is 160 layers above the bottom of tprogs which ends up giving 160-20 is layer 140 or 139 for 0 based

## LPF/UPW package

In [ ]:
hk = np.zeros(botm.shape)
vka = np.zeros(botm.shape)
sy = np.zeros(botm.shape)
ss = np.zeros(botm.shape)

In [ ]:
fig,ax=plt.subplots(figsize=(8,8))
# plt.imshow(K[0,:,:])
# plt.imshow(K[-1,:,:])
plt.imshow(K[:,0,:],aspect=1/10)

In [ ]:
from scipy.stats import hmean, gmean
Kx_upscaled = np.zeros((num_tprogs,nrow,ncol))
Kz_upscaled = np.zeros((num_tprogs,nrow,ncol))
Sy_upscaled = np.zeros((num_tprogs,nrow,ncol))
Ss_upscaled = np.zeros((num_tprogs,nrow,ncol))

for k in np.arange(1,num_tprogs+1):
    # calculate upscale from bottom up
    Kx_upscaled[-k,:,:] = np.nanmean(K[(-k*upscale):(-k*upscale-upscale):-1,:,:],axis=0)
    Kz_upscaled[-k,:,:] = hmean(K[(-k*upscale):(-k*upscale-upscale):-1,:,:],axis=0)
    Sy_upscaled[-k,:,:] = np.nanmean(Sy[(-k*upscale):(-k*upscale-upscale):-1,:,:],axis=0)
    Ss_upscaled[-k,:,:] = np.nanmean(Ss[(-k*upscale):(-k*upscale-upscale):-1,:,:],axis=0)

In [ ]:
# fig,ax=plt.subplots(2,4,figsize=(8,4))
# for n in np.arange(0,4):
#     ax[0,n].imshow(Kx_upscaled[n*4,:,:])
#     ax[1,n].imshow(Kz_upscaled[n*4,:,:])


In [ ]:
# # take of 2 for the bottom layers and 1 for the unsat zone layer up top
# # for tprogs arrays 0 is the bottom of the model, so flipping on z will fix
# hk[1:-2,:,:] = np.flip(K[-hk.shape[0]+1:-2,:,:],axis=0)
# sy[1:-2,:,:] = np.flip(Sy[-sy.shape[0]+1:-2,:,:],axis=0)
# ss[1:-2,:,:] = np.flip(Ss[-ss.shape[0]+1:-2,:,:],axis=0)
hk[1:-2,:,:] = Kx_upscaled
vka[1:-2,:,:] = Kz_upscaled
sy[1:-2,:,:] = Sy_upscaled
ss[1:-2,:,:] = Ss_upscaled

The LPF package should use the means for the complete TPROGs layers available for the first layer

In [ ]:
top = m.dis.top.array
bot1 = m.dis.botm.array[0,:,:]
# set parameters based on upscaled unsaturated zone
hk[0,:,:] = np.mean(tc.get_tprogs_for_elev(K, top, bot1,tprogs_info),axis=0)
vka[0,:,:] = hmean(tc.get_tprogs_for_elev(K, top, bot1,tprogs_info),axis=0)
sy[0,:,:] = np.mean(tc.get_tprogs_for_elev(Sy, top, bot1,tprogs_info),axis=0)
ss[0,:,:] = np.mean(tc.get_tprogs_for_elev(Ss, top, bot1,tprogs_info),axis=0)



In [ ]:
rows,cols = grid_p.row.values-1, grid_p.column.values-1

In [ ]:
K.shape
# top-bot1
K[tc.elev_to_tprogs_layers(bot1, tprogs_info)[8,25], 8, 25]

# np.isnan(K[166,:,:]).sum()

Trying a VANI of 10 for the tprogs units to allow some vertical gradient forcing

In [ ]:
# check proportions of hydrofacies in TPROGs realization
tprogs_hist = np.histogram(masked_tprogs, [0,1.1,2.1,3.1,4.1])[0]
tprogs_hist = tprogs_hist/np.sum(tprogs_hist)

# scale vertical conductivity with a vertical anisotropy factor based
# on quantiles in the upscaled tprogs data
for n, p in enumerate(np.arange(1,5)):
    vka[vka >np.quantile(vka, (1-tprogs_hist[n]))] /= params.vani[p]

The tuff breccia is very dense, hard and low water yielding. It is supposedly responsible for the many "haystack" hills in the eastern part of the county

DWR report has a few final well pumping rates, drawdowns and specific capacities but limited.

Fleckenstein et al. 2006 found the Mehrten had  
Kh = 1 to 1.8 x10^-5 m/s  
Kv = 1 to 1.8 x10^-7 m/s  
vani ~= 100  
Sy = 0.15 to 0.2  
Ss = 1e-4 to 1e-3 m^-1  

In [ ]:
# plt.imshow(vka[0,:,:])
# plt.show()
# plt.imshow(hk[0,:,:])
# plt.show()
# plt.imshow(deep_geology[0,:,:])
# plt.show()


In [ ]:
# set values for second to bottom layer, Laguna formation
hk[-2,:,:] = params.loc[5,'K_m_d']
vka[-2,:,:] = params.loc[5,'K_m_d']/params.loc[5,'vani'] 
sy[-2,:,:] = params.loc[5,'Sy']
ss[-2,:,:] = params.loc[5,'Ss']

# this is causing potentially high water levels in the foothills
# the deep_geology array shows where the mehrten formation comes out of the surface
hk[deep_geology[:,:,:].astype(bool)] = params.loc[6,'K_m_d']
vka[deep_geology[:,:,:].astype(bool)] = params.loc[6,'K_m_d']/params.loc[6,'vani'] 
sy[deep_geology[:,:,:].astype(bool)] = params.loc[6,'Sy']
ss[deep_geology[:,:,:].astype(bool)] = params.loc[6,'Ss']

# set values for bottom layer, Mehrten formation
hk[-1,:,:] = params.loc[6,'K_m_d']
vka[-1,:,:] = params.loc[6,'K_m_d']/params.loc[6,'vani'] 
sy[-1,:,:] = params.loc[6,'Sy']
ss[-1,:,:] = params.loc[6,'Ss']

In [ ]:
# because I need to maintain a confined sytem for now, I should use the storativity option because
# the gravel has a very low specific storage that might be causing the large jumps
# in groundwater elevation

# may need to adjust to use specific yield only gravel and sand

In [ ]:
def calc_storativity(m, ss, sy):
    thickness = np.zeros((m.dis.nlay,m.dis.nrow,m.dis.ncol))
    thickness[0,:,:] = m.dis.top.array - m.dis.botm.array[0,:,:]
    thickness[1:,:,:] =  np.diff(m.dis.botm.array,axis=0)*-1
    # calculate storativity based on layer thickness
    storativity = thickness*ss
    # add specific yield to specific storage 
    storativity += sy
    return(storativity)

storativity = calc_storativity(m, ss, sy)

layer_cutoff = 3
storativity[layer_cutoff:,:,:] -= sy[layer_cutoff:,:,:]

# anything deeper than layer 3 try with just Ss because the wells aren't showing enough signal
# avg of layer 3 is -14m so 50 ft, but confinement most likely starts around 100ft(layer 7) so lets adjust to that
# using just Ss below 3 layers shows too much signal again, I added back Sy, but lowered Sy values by around 0.05-0.1
# using layer 7 meant most wells were very flat so I increased Ss by 1 order of magnitude and switched back to
# a layer 3 cutoff

np.savetxt(model_ws+'/input_data/storativity.tsv', np.reshape(storativity,(nlay*nrow, ncol)),
          delimiter=' ', fmt='%.6e')

In [ ]:
coarse = (masked_tprogs==1)|(masked_tprogs==2)

In [ ]:
# layvka 0 means vka is vert K, non zero means its the anisotropy ratio between horiz and vert
layvka = 0

# LAYTYP MUST BE GREATER THAN ZERO WHEN IUZFOPT IS 2
# 0 is confined, >0 convertible, <0 convertible unless the THICKSTRT option is in effect
# laytyp = np.ones(nlay)  
# laytyp = np.zeros(nlay)
# try making first 10 layers convertible/ unconfined, model crashes trying to convert wet/dry
num_unconf = 1
laytyp = np.append(np.ones(num_unconf), np.zeros(nlay-num_unconf))

# Laywet must be 0 if laytyp is confined laywet = [1,1,1,1,1]
laywet = np.zeros(len(laytyp))
laywet[laytyp==1] = 1
#ipakcb = 55 means cell-by-cell budget is saved because it is non zero (default is 53)

# gel = flopy.modflow.ModflowUpw(model = m, hk =hk, layvka = layvka, vka = vka, 
#                                sy=sy, ss=ss,
#                             laytyp=laytyp, laywet = 0, ipakcb=55) # laywet must be 0 for UPW

gel = flopy.modflow.ModflowLpf(model = m, hk =hk, layvka = layvka, vka = vka, sy=sy, 
#                                ss = storativity, storagecoefficient=True, #storativity
                               ss=ss, 
                               laytyp=laytyp, laywet = laywet, ipakcb=55)

In [ ]:
gel_nam = pd.Series(['LPF','UPW'])[pd.Series(['LPF','UPW']).isin(m.get_package_list())].values[0]
gel = getattr(m,gel_nam)
frac = [0, 0.582, 0.159, 0.203, 0.056]
frac = np.cumsum(frac)
print('TPROGs fractions/quantiles: ',frac)
for n in ['hk','vka','sy','ss']:
    q = np.quantile(getattr(gel,n).array,frac)
    print(n,  list('{:8.2E}'.format(qn, '.2E') for qn in q))
# # np.histogram?

In [ ]:
# gel.write_file()

In [ ]:
gel.check()
# option to make switching between packages easier
# 'LPF' in m.get_package_list()

## Review geologic parameters

In [ ]:
plt_dir = gwfm_dir+'/Levee_setback/figures/'

In [ ]:

lays2plt = [0,1,4,8,14,18,19,20]
plt_cols = 4
fig,ax=plt.subplots(int(len(lays2plt)/plt_cols),plt_cols,figsize=(12,4),sharex='col',sharey='row')
fig.tight_layout()

for n,k in enumerate(lays2plt):
    ax_n = ax[int(n / plt_cols), n % plt_cols]
    im = ax_n.imshow(hk[k,:,:], norm = mpl.colors.LogNorm(vmin = hk.min(), vmax=hk.max()))
    ax_n.set_title('Layer '+str(k))
hcb = plt.colorbar(im, shrink = 0.5,ax=ax)
hcb.set_label('Horizontal Conductivity (m/d)')
plt.savefig(plt_dir+'Horizontal_conductivity_select_layers.png',dpi=600,bbox_inches='tight')

# plt.colorbar(mappable=im)
print('HK of Mehrten and Laguna: ',np.unique(hk[19,:,:]))
print('Layer 1 range: ', np.histogram(hk[0,:,:],4)[1])
print('Layer 5 range: ',np.histogram(hk[4,:,:],4)[1])

### Scaling factors for GHB, RIV, UZF/RCH and WEL package

In [ ]:
gel_dir = gwfm_dir+'/UPW_data'
if 'GHB_UZF_WEL_scaling.csv' in os.listdir(model_ws):
    scaling_factors = pd.read_csv(model_ws+'/GHB_UZF_WEL_scaling.csv',delimiter = ',')
else:
    scaling_factors = pd.read_csv(gel_dir+'/GHB_UZF_WEL_scaling.csv',delimiter = ',')
    scaling_factors.to_csv(model_ws+'/GHB_UZF_WEL_scaling.csv')

scaling_factors_all = scaling_factors.copy()
# convert from K (m/s) to K (m/d)
scaling_factors_all.loc[scaling_factors_all.GroupName.isin(['GHB']),'StartValue']*=86400


scaling_factors = scaling_factors.set_index('ParamName')['StartValue']

## Import soil data for Lake Package, UZF Package, SFR Package hydraulic parameters

In [ ]:
mb_name = gwfm_dir+"/DIS_data/NewModelDomain/GWModelDomain_52_9deg_UTM10N_WGS84.shp"

mb = gpd.read_file(mb_name)
mb = mb.to_crs('epsg:32610')

### Write grid_uzf to shapefile to avoid having to repeat analysis

In [ ]:
uzf_path = gwfm_dir+'\\UZF_data'


In [ ]:
# grid_uzf = gpd.read_file(uzf_path+'/final_grid_uzf/griduzf.shp')

In [ ]:
soilKs_array = np.loadtxt(uzf_path+'/final_soilKs.tsv', delimiter = '\t')
soiln_array = np.loadtxt(uzf_path+'/final_soiln.tsv', delimiter = '\t')
soileps_array = np.loadtxt(uzf_path+'/final_soileps.tsv', delimiter = '\t')
soildepth_array = np.loadtxt(uzf_path+'/final_soildepth.tsv', delimiter = '\t')



# SFR

In [ ]:
sfr_dir = gwfm_dir+'/SFR_data/'

## Adjust Blodgett Dam scenario here

In [ ]:
# scenario = 'dam'
# scenario = 'actual'
# scenario = 'new'
scenario = 'none'

In [ ]:
grid_sfr = gpd.read_file(sfr_dir+'/final_grid_sfr/grid_sfr.shp')


In [ ]:
XS8pt = pd.read_csv(sfr_dir+'8pointXS.csv')
# XSlocs = gpd.read_file(sfr_dir+'8pointXS_locs/8pointXS_locs.shp')
# new shapefile with an extra point for blodgett dam as site 16.5
XSlocs = gpd.read_file(gwfm_dir+'/Blodgett_Dam/geospatial/8pointXS_locs/8pointXS_locs.shp')
XSlocs.crs = 32610

XSg  = gpd.sjoin(grid_sfr, XSlocs, how = "inner", op= "contains", lsuffix = 'sfr',rsuffix = 'xs')
# print(len(XSg))

# # Append the grid_breach location to the list of cross sections to split the segment
# XSg = XSg.append(grid_breach).sort_values('reach')
# # Copy the XS site name from the previous last site to the breach site to keep same XS
# XSg.Site.iloc[-1] = XSg.Site.iloc[-2]
# len(XSg), len(XS8pt.loc[0,:])/2

if scenario == 'none':
    # if no blodgett dam scenario then remove the extra cross section
    XSg = XSg.loc[(XSg.Site!=16.5)]
    XSg = XSg.loc[(XSg.Site!=16.2)]
#     XSg = XSg.loc[XSg.Site!=16.2]
elif scenario == 'actual':
    XSg_side = XSg.loc[XSg.Site==16.5]
    XSg_side.loc[:,'Site'] = 16.4
    XSg = XSg.append(XSg_side)
elif scenario == 'design':
    # may or may not want to remove the segment before
    XSg = XSg.loc[(XSg.Site!=16.2)]

# if the scneario is the restructured or designed dam then no change in the segments is necessary
# sort by site to make sure any XS added are properly included
XSg = XSg.sort_values('Site')
# print(len(XSg))


In [ ]:
XSg['iseg'] = np.arange(2,len(XSg)+2) # add the segment that corresponds to each cross section

In [ ]:

# if scenario == 'dam':
#     # designed scenario flow through dam only
#     new_xs = pd.read_csv(gwfm_dir+'/Blodgett_Dam/geospatial/02_designed_XS.csv', skiprows=1)
# elif scenario =='actual':
#     # current situation, flow around dam and after dam
#     new_xs = pd.read_csv(gwfm_dir+'/Blodgett_Dam/geospatial/03_actual_XS.csv', skiprows=1)
# elif scenario =='new':
#     # depending scenario, use different input cross sections for 16.5
#     new_xs = pd.read_csv(gwfm_dir+'/Blodgett_Dam/geospatial/01_New_wide_XS.csv')

# if there is a scneario then need to add the new XS
if scenario != 'none':
    XS8pt = pd.concat([XS8pt,new_xs],axis=1)

some of the XS are not lining up with grid_sfr so they aren't being connected. Need to fix this and also look at how many XS are really needed to capture the change in river morphology

### Read in 8 pt XS, revised by simplifying from Constantine 2001

In [ ]:
# There is one reach for each cell that a river crosses
NSTRM = -len(grid_sfr)
# There should a be a stream segment if there are major changes
# in variables in Item 4 or Item 6
# 1st segment is for the usgs Michigan Bar rating curve, one for each XS, plus 2 for the floodplain diversion
NSS = 1 + len(XSg) 
# NSS = 2
# nparseg (int) number of stream-segment definition with all parameters, must be zero when nstrm is negative
NPARSEG = 0
CONST = 86400 # mannings constant for SI units, 1.0 for seconds, 86400 for days
# real value equal to the tolerance of stream depth used in
# computing leakage between each stream reach and active model cell
DLEAK = 0.0001 # unit in lengths, 0.0001 is sufficient for units of meters
IPAKCB = 55
# writes out stream depth, width, conductance, gradient when cell by cell
# budget is specified and istcb2 is the unit folder
ISTCB2 = 54
# isfropt = 1 is no unsat flow
# specifies whether unsat flow beneath stream or not, isfropt 2 has properties read for each reach, isfropt 3 also has UHC
# read for each reach, isfropt 4 has properties read for each segment (no UHC), 5 reads for each segment with UHC
ISFROPT = 1
# nstrail (int), number of trailing weave increments used to represent a trailing wave, used to represent a decrease 
# in the surface infiltration rate. Can be increased to improve mass balance, values between 10-20 work well with error 
# beneath streams ranging between 0.001 and 0.01 percent, default is 10 (only when isfropt >1)
NSTRAIL = 20
# isuzn (int) tells max number of vertical cells used to define the unsaturated zone beneath a stream reach (default is 1)
ISUZN = 1
#nsfrsets (int) is max number of different sets of trailing waves (used to allocate arrays), a value of 30 is sufficient for problems
# where stream depth varies often, value doesn't effect run time (default is 30)
NSFRSETS = 30
# IRTFLG (int) indicates whether transient streamflow routing is active, must be specified if NSTRM <0. If IRTFLG >0 then
# flow will be routed with the kinematic-wave equations, otherwise it should be 0 (only for MF2005), default is 1
IRTFLG = 1
# numtim (int) is number of sub time steps used to route streamflow. Streamflow time step = MF Time step / NUMTIM. 
# Default is 2, only when IRTFLG >0
NUMTIM = 4
# weight (float) is a weighting factor used to calculate change in channel storage 0.5 - 1 (default of 0.75) 
WEIGHT = 0.75
# flwtol (float), flow tolerance, a value of 0.00003 m3/s has been used successfully (default of 0.0001)
# 0.00003 m3/s = 2.592 m3/day
# a flow tolerance of 1 cfs is equal to 2446.57 m3/day
# if my units are in m3/day then flwtol should be in m3/day
FLWTOL = 3


sfr = flopy.modflow.ModflowSfr2(model = m, nstrm = NSTRM, nss = NSS, nparseg = NPARSEG, 
                           const = CONST, dleak = DLEAK, ipakcb = IPAKCB, istcb2 = ISTCB2, 
                          isfropt = ISFROPT, nstrail = NSTRAIL, isuzn = ISUZN, irtflg = IRTFLG, 
                          numtim = NUMTIM, weight = WEIGHT, flwtol = FLWTOL,
                                reachinput=True, transroute=True, tabfiles=True,
                                tabfiles_dict={1: {'numval': nper, 'inuit': 56}})

In [ ]:
# Add option block at the top of the sfr input file for tabfiles
tab_option = flopy.utils.OptionBlock(options_line = ' reachinput transroute tabfiles 1 ' + str(nper), package = sfr, block = True)
sfr.options = tab_option
# sfr.options

In [ ]:
# Modflow NWT additions to SFR package set up
# sfr.transroute = True
# sfr.reachinput = True
# sfr.tabfiles = True
# # numval is the number of values in the flow tab files, inuit is the corresponding unit file
# sfr.tabfiles_dict = {1: {'numval': nper, 'inuit': 56}}

In [ ]:
xs_sfr = grid_sfr.set_index('reach')
# set all reaches to start as segment 1 which will be changed iteratively based on the number of cross-sections
xs_sfr['iseg'] = 1
# add a column reach_new that will be changed iteratively as the segment number is changed
xs_sfr['reach_new'] = xs_sfr.index
# xs_sfr

## Define reach data based on ISFROPT

In [ ]:
## Given the reach number of each XS, the 718 reaches will be broken down into each segment
## create a new reach column based on XS reach number and 

# segcount = 2
for i in np.arange(0,len(XSg)):
    temp_reach = XSg.reach.values[i]
    rchnum = xs_sfr.index[-1] - temp_reach+1
    xs_sfr.reach_new.loc[temp_reach:] = np.linspace(1,rchnum, rchnum)
#     xs_sfr.iseg.loc[temp_reach:] = segcount
    xs_sfr.iseg.loc[temp_reach:] = XSg.iseg.values[i]
#     segcount +=1
    
# for simple 1 XS model
# temp_reach = XSg.reach
# rchnum = xs_sfr.index[-1] - temp_reach+1
# xs_sfr.reach_new.loc[temp_reach:] = np.linspace(1,rchnum, rchnum)
# xs_sfr.iseg.loc[temp_reach:] = segcount
# segcount +=1

In [ ]:
xs_sfr.reach_new = xs_sfr.reach_new.astype(int)


In [ ]:
sfr_rows = (grid_sfr.row.values-1).astype(int)
sfr_cols = (grid_sfr.column.values-1).astype(int)
# Determine which layer the streamcell is in
# since the if statement only checks whether the first layer is greater than the streambed elevation, 
sfr_lay = get_layer_from_elev((grid_sfr.z.values-1), botm[:, sfr_rows, sfr_cols], m.dis.nlay)


In [ ]:
# KRCH, IRCH, JRCH, ISEG, IREACH, RCHLEN, STRTOP, SLOPE, STRTHICK, STRHC1, THTS, THTI, EPS, UHC

columns = ['KRCH', 'IRCH', 'JRCH', 'ISEG', 'IREACH', 'RCHLEN', 'STRTOP', 
               'SLOPE', 'STRTHICK', 'STRHC1', 'THTS', 'THTI', 'EPS', 'UHC']

sfr.reach_data.node = grid_sfr.index
sfr.reach_data.k = sfr_lay.astype(int)
sfr.reach_data.i = sfr_rows
sfr.reach_data.j = sfr_cols
sfr.reach_data.iseg = xs_sfr.iseg
sfr.reach_data.ireach = xs_sfr.reach_new
sfr.reach_data.rchlen = xs_sfr.length_m.values
sfr.reach_data.strtop = grid_sfr.z.values-1
sfr.reach_data.slope = grid_sfr.slope.values
 # a guess of 2 meters thick streambed was appropriate
sfr.reach_data.strthick = soildepth_array[sfr.reach_data.i, sfr.reach_data.j]

# UZF parameters
sfr.reach_data.thts = soiln_array[sfr.reach_data.i, sfr.reach_data.j]/100
sfr.reach_data.thti = sfr.reach_data.thts
sfr.reach_data.eps = soileps_array[sfr.reach_data.i, sfr.reach_data.j]
sfr.reach_data.uhc = vka[0,sfr.reach_data.i, sfr.reach_data.j]


In [ ]:
grid_sfr['dist_m'] = grid_sfr.length_m.cumsum()
grid_sfr.dist_m -= grid_sfr.dist_m.iloc[0]

## Could try TPROGs alternative 

In [ ]:
topbotm = np.zeros((m.dis.nlay+1,m.dis.nrow,m.dis.ncol))
topbotm[0,:,:] = m.dis.top.array
topbotm[1:,:,:] = m.dis.botm.array

In [ ]:
strbd_thick = 4
top = m.dis.top.array.copy()
# use values of stream top instead of DEM top
top[sfr_rows, sfr_cols] = grid_sfr.z.values
bot_str_arr = top - strbd_thick
# get_tprogs_for_elev(K, m_c.dis.top.array, m_c.dis.top.array- np.linspace(1,4,m_c.dis.ncol), rows = sfr_rows, cols = sfr_cols)
strbd_tprogs = tc.get_tprogs_for_elev(K, top, bot_str_arr, tprogs_info,rows = sfr_rows, cols = sfr_cols)
sfr_K = gmean(strbd_tprogs,axis=0)/100 # divide by 100 to ease convergence, but variability is still there
plt.plot(sfr_K)
plt.ylabel('VKA (m/d)')
plt.xlabel('Reach')
# temp fix to get convergence
# sfr_K = 0.01

In [ ]:
# set stream hydraulic conductivity based on soil maps
# sfr.reach_data.strhc1 = soilKs_array[sfr.reach_data.i, sfr.reach_data.j]*scalingfactors.RIV
# set hydraulic conductivity smaller than aquifer hydraulic conductivity to limit interaction
# and ease the numerical stress
sfr.reach_data.strhc1 = sfr_K

# calibration of the whole river now by scaling conductivity
m.sfr.reach_data.strhc1 = m.sfr.reach_data.strhc1
# next step is to break river up into reaches based on the grain size analysis or perhaps just by stream segment

The range of of maximum log(river bed K) (m/s) from Fleckenstein's cosumnes paper was -4.0 to -4.8 which gives VKA = 1.6e-5 to 1E-4 m/s and 1.38 - 8.64 m/day. Minimim of 1E-7 m/s or 8.64E-3 m/day. The mean was around 0.17 m/day

In [ ]:
# sfr.write_file()

With no sfr K reduction, the model failed to converge in stress period 12 time step 3. With sfr K reduced by a factor of 1/100 the model ran to stress period 1246 time step 4 while the previous model with low K went to stress period 2026 time step 4. Dividing by 1000 actually caused the model to fail to converge at stress period 100 which shows that there was not enough water coming in to the aquifer from the stream, and 

In [ ]:
mb4rl = pd.read_csv(sfr_dir+'michigan_bar_icalc4_data.csv', skiprows = 0, sep = ',')
# mb4rl.plot(x='gage_height_va',y='discharge_va', legend = False)
# plt.xlabel('Gage height (m)')
# plt.ylabel('Discharge $(m^3/d$)')
# plt.ticklabel_format(style='scientific') # plain to show all zeros
# plt.title('Simplified USGS Michigan Bar Rating Curve')
# plt.savefig('Plots/Model_SFR_UZF_Progress/MB_ratingcurve', dpi = 300, bbox_inches='tight')

## Define segment data

Median grain size (mm) ranges from 1 mm – 30 mm along surveyed sites, which gives a range of 0.026-0.035 for a stable channel
Moderate channel irregularity due to channel scouring and pools alternating, range of 0.006-0.010
Gradual cross section change: 0.000 adjustment
Effect of obstructions: minor due to occasional downed logs and debris in river, 0.005-0.015
Amount of vegetation: large on banks due to willows and cottonwood trees, 0.025-0.050, and negligible in the channel
Degree of meandering: minor due to levees, m = 1.0

n = (nb+n1+n2+n3+n4)*m (b=base,1=surface irregularity, 2 = XS variation, 3 = obstructions, 4 = vegetation, m = correction for meandering)
n = (0.03+0.08+0.01) = 0.048 in channel
n = (0.048 +0.03) = 0.078 on banks


In [ ]:
sfr_seg = sfr.segment_data[0].copy()

In [ ]:
# There is one dictionary key for each stress period (starting at 0) and in each dictionary key there is a 
# rec array holding an entry for each stream segment where nseg shows which segment it is (ie no dictionary key for segment)
# If ITMP = 0 Item 4 is used, if ITMP >0 Item 6 is used, 
# if ITMP <0 the stream segment data not defined in Item 4 will be reused form the last stress period

# Define stress period data need one for each stress period
# Dataset 5 will be built automatically from segment_data unless specified
# ITMP (int) for reusing or reading stream seg data that can change each stress period
#IRDFLG, 0 is input data printed, greater than 0 input data is not printed
# doesn't seem to change the value
# IPTFLG, 0 is streamflow-routing results printed, greater than 0 not

In [ ]:
# 15.0=14, 16.2 = 15, 16.4 = 16, 16.5 = 17, 17.0=18
# 14 outseg will be the side channel (16), 15 is the diversion before the Dam from 14 iupseg
# outseg for 15 will be -1 for the lake representing BLodgett Dam
# there is a diversion from 15 (segment to Dam) to 16 (side channel) to correct for the flood diversion
# so that below 500 cfs flow only goes to the side channel and above 500 cfs flow is 80% to Dam and 20% to side channel
# based on the idea that the side channel has a XS roughly 1/4 the size of the main channel and under high flows there
# will be more depth and force that flow will most likely be dominantly straight and avoid the side channel more

# if scenario =='actual':
#     pre_seg = XSg.loc[XSg.Site==16.2,'iseg'].iloc[0]
#     side_seg = XSg.loc[XSg.Site==16.4,'iseg'].iloc[0]
# elif (scenario =='actual') | (scenario=='design'):
#     post_seg = XSg.loc[XSg.Site==16.5,'iseg'].iloc[0]
# print(pre_seg,side_seg,post_seg)

In [ ]:
# alternate version of segment data loading using if statements when filtering data rather than in a loop
sfr_seg.nseg = np.arange(1,NSS+1)

sfr_seg.icalc = 2 # Mannings and 8 point channel XS is 2 with plain MF, 5 with SAFE
sfr_seg.icalc[0] = 4 # use stage, discharge width method for Michigan Bar (nseg=1)
sfr_seg.nstrpts[sfr_seg.icalc==4] = len(mb4rl) # specify number of points used for flow calcs
sfr_seg.outseg = sfr_seg.nseg+1 # the outsegment will typically be the next segment in the sequence
sfr_seg.iupseg = 0 # iupseg is zero for no diversion
# correct outseg and iupseg to account for Blodgett Dam scenario
# if scenario =='design':
#     sfr_seg.outseg[sfr_seg.nseg==post_seg-1]=-1 # segment before dam flows to lake
#     sfr_seg.iupseg[sfr_seg.nseg==post_seg]=-1 # lake outflow is diverted to segment after dam
# elif scenario == 'actual':
#     sfr_seg.outseg[sfr_seg.nseg==pre_seg-1] = side_seg # the river should flow to the side segment first
#      # there will be a diversion from the river to the dam above 500 cfs, of which 20% will be returned to the side channel
#     sfr_seg.iupseg[sfr_seg.nseg==pre_seg] = pre_seg-1
#     sfr_seg.iprior[sfr_seg.nseg==pre_seg] = -3 # iprior=-3 any flows above the flow specified will be diverted
#     sfr_seg.flow[sfr_seg.nseg==pre_seg] = 500*0.3048*86400 # 500 cfs is the start of higher flow in the Cosumnes
#     sfr_seg.outseg[sfr_seg.nseg==pre_seg] = -1 #outflow from short segment before Dam is the LAK for the dam

#     # adjust for flow from pre dam segment back to side channel
#     sfr_seg.iupseg[sfr_seg.nseg==side_seg] = pre_seg
#     sfr_seg.iprior[sfr_seg.nseg==side_seg] = -2 # the flow diverted is a % of the total flow in the channel
#     sfr_seg.flow[sfr_seg.nseg==side_seg] = 0.2 # the side channel is about 1/4 the size so 20% of flow should run through
#     # divert flow from lake back into the segment after the dam
#     sfr_seg.iupseg[sfr_seg.nseg==post_seg] = -1 # no need to change iprior because diversion is based on lake stage
    
# set a flow into segment 1 for the steady state model run
sfr_seg.flow[0] = 2.834*86400. # m3/day, originally 15 m3/s
# set the values for ET, runoff and PPT to 0 as the inflow will be small relative to the flow in the river
sfr_seg.runoff = 0.0
sfr_seg.etsw = 0.0
sfr_seg.pptsw = 0.0

# Manning's n data comes from Barnes 1967 UGSS Paper 1849 and USGS 1989 report on selecting manning's n
# RoughCH is only specified for icalc = 1 or 2
sfr_seg.roughch[(sfr_seg.icalc==1) | (sfr_seg.icalc==2)] = 0.048
# ROUGHBK is only specified for icalc = 2
sfr_seg.roughbk[(sfr_seg.icalc==2) | (sfr_seg.icalc==5)] = 0.083# higher due to vegetation

In [ ]:
sfr.segment_data[0] = sfr_seg

In [ ]:
# Pull out data for upstream and downstream reach of each segment
up_data = xs_sfr.drop_duplicates('iseg')
dn_data = xs_sfr.sort_values('reach_new',ascending = False).drop_duplicates('iseg').sort_values('iseg')


# Need to return to later and remove hard coding
# These are getting used for initial guesses
# Read in first stress period when ICALC = 1 or 2 and ISFROPT is 5
# Dataset 6b
sfr.segment_data[0].hcond1 = sfr.reach_data.strhc1[0]
sfr.segment_data[0].thickm1 = 2
sfr.segment_data[0].elevup = up_data.z.values
sfr.segment_data[0].width1 = 20
sfr.segment_data[0].depth1 = 1
sfr.segment_data[0].thts1 = 0.4
sfr.segment_data[0].thti1 = 0.15
sfr.segment_data[0].eps1 = 4
sfr.segment_data[0].uhc1 = sfr.reach_data.strhc1[0]

# Dataset 6c
sfr.segment_data[0].hcond2 = sfr.reach_data.strhc1[-1]
sfr.segment_data[0].thickm2 = 2
sfr.segment_data[0].elevdn = dn_data.z.values
sfr.segment_data[0].width2 = 20
sfr.segment_data[0].depth2 = 1
sfr.segment_data[0].thts2 = 0.4
sfr.segment_data[0].thti2 = 0.15
sfr.segment_data[0].eps2 = 4
sfr.segment_data[0].uhc2 = sfr.reach_data.strhc1[-1]


In [ ]:
# Change column name to float type for easier referencing in iteration
XS8pt.columns = XS8pt.columns.astype('float')
# Pre-create dictionary to be filled in loop
sfr.channel_geometry_data = {0:{j:[] for j in np.arange(2,len(XSg)+2)}  }

xsnum = 2
for k in XSg.Site.values:
        pos = int(XS8pt.columns.get_loc(k))
        XCPT = XS8pt.iloc[:,pos].values
        ZCPT = XS8pt.iloc[:,pos+1].values
        ZCPT_min = np.min(ZCPT)
        ZCPT-= ZCPT_min
        sfr.channel_geometry_data[0][xsnum] = [XCPT, ZCPT]
        xsnum += 1



In [ ]:
FLOWTAB = mb4rl.discharge_va.values
DPTHTAB = mb4rl.gage_height_va.values
WDTHTAB = mb4rl.chan_width.values
sfr.channel_flow_data = {0: {1: [FLOWTAB, DPTHTAB, WDTHTAB]}}

In [ ]:
# sfr.plot_path(start_seg=1, end_seg=0, plot_segment_lines=True)
# plt.savefig('Plots/Model_SFR_UZF_Progress/sfr_elev_vs_model_top.png', dpi = 600, bbox_inches='tight')

### Tab Files for SFR

In [ ]:
# For the tab files the left column is time (in model units) and the right column is flow (model units)
# Time is days, flow is cubic meters per day
# USGS presents flow in cfs (cubic feet per second)
inflow = pd.read_csv(sfr_dir+'MB_daily_flow_cfs_2010_2019.csv', index_col = 'datetime', parse_dates = True)

# filter out data between the stress period dates
inflow = inflow.loc[strt_date:end_date]
# covnert flow from cubic feet per second to cubic meters per day
inflow['flow_cmd'] = inflow.flow_cfs * (86400/(3.28**3))

# # np.arange(0,len(flow_cmd))

time_flow = np.vstack((np.arange(time_tr0,len(inflow.flow_cmd)+time_tr0),inflow.flow_cmd))
time_flow = np.transpose(time_flow)
# add a first row to account for the steady state stress period
# median instead of mean because of too much influence from large values
if no_ss == False:
    time_flow = np.row_stack(([0, inflow.flow_cmd.median()], time_flow))

np.savetxt(model_ws+'/MF.tab',time_flow, delimiter = '\t')

In [ ]:
# inflow.flow_cfs.hist(bins = [0,150,1500, 3000,10000,20000])
# fig,ax=plt.subplots(figsize=(8,4))
# inflow.plot(y='flow_cfs',legend=False,ax=ax)
# plt.ylabel('Streamflow (cfs)',size=16)
# plt.xlabel('Date',size=16)
# plt.savefig(plt_dir+'streamflow_michigan_bar.png',dpi=600,bbox_inches='tight')

In [ ]:
flopy.modflow.mfaddoutsidefile(model = m, name = 'DATA',extension = 'tab',unitnumber = 56)

"When subsurface recharge (MBR2) is negligible,
stream runoff at the mountain front (runoff measured at
point B in Figure 1, or RO) may be considered the total
contribution to MFR [Anderholm, 2000]." (Wilson and Guan 2004)

## GHB NW, SE set up

In [ ]:
# join top and botm for easier array referencing for elevations
top_botm = np.zeros((m.dis.nlay+1,m.dis.nrow,m.dis.ncol))
top_botm[0,:,:] = m.dis.top.array
top_botm[1:,:,:] = m.dis.botm.array


In [ ]:
# ghb_hk_nw = scaling_factors.K_nw
# ghb_hk_se = scaling_factors.K_se

distance = 5000
t0 = time.time()

col_strt=1

ghbnw_spd = pd.DataFrame(np.zeros((np.sum(nlay-nw_lay[col_strt:]),5)))
ghbnw_spd.columns = ['k','i','j','bhead','cond']
# start 1 column in to avoid conflict with delta boundary

# get all of the j,k indices to reduce math done in the for loop
yz = np.zeros((np.sum(nlay-nw_lay[col_strt:]),2)).astype(int)
n=0
for j in np.arange(col_strt,ncol):
    for k in np.arange(nw_lay[j], nlay):
        yz[n,0] = j
        yz[n,1] = k
        n+=1
        
condnw = hk[yz[:,1],0,yz[:,0]]*(top_botm[yz[:,1],0,yz[:,0]]-top_botm[yz[:,1]+1,0,yz[:,0]])*delr/distance
# condnw = ghb_hk_nw*(top_botm[yz[:,1],0,yz[:,0]]-top_botm[yz[:,1]+1,0,yz[:,0]])*delr/distance

ghbnw_spd.cond = condnw
ghbnw_spd.bhead = avg_nw[yz[:,0]]
ghbnw_spd.k = yz[:,1]
ghbnw_spd.j = yz[:,0]
ghbnw_spd.i = 0

ghbse_spd = pd.DataFrame(np.zeros((np.sum(nlay-se_lay[col_strt:]),5)))
ghbse_spd.columns = ['k','i','j','bhead','cond']

# get all of the j,k indices to reduce math done in the for loop
yz = np.zeros((np.sum(nlay-se_lay[col_strt:]),2)).astype(int)
n=0
for j in np.arange(col_strt,ncol):
    for k in np.arange(se_lay[j], nlay):
        yz[n,0] = j
        yz[n,1] = k
        n+=1
condse = hk[yz[:,1],int(nrow-1),yz[:,0]]*(top_botm[yz[:,1],-1,yz[:,0]]-top_botm[yz[:,1]+1,-1,yz[:,0]])*delr/distance
# condse = ghb_hk_se*(top_botm[yz[:,1],-1,yz[:,0]]-top_botm[yz[:,1]+1,-1,yz[:,0]])*delr/distance

ghbse_spd.cond = condse
ghbse_spd.bhead = avg_se[yz[:,0]]
ghbse_spd.k = yz[:,1]
ghbse_spd.j = yz[:,0]
ghbse_spd.i = nrow-1
        
resample_time = time.time() - t0
print("GHB time: {:.3f} sec".format(time.time() - t0))

In [ ]:
# only need to recalculate conductance in ucode
def recalc_cond(i,j,k,hk):
    distance = 5000
    delr = m.dis.delr.array.mean()
    cond = hk*(top_botm[k,i,j]-top_botm[k+1,i,j])*delr/distance
    return(cond)


## Southwest GHB boundary (specified head for outflow to the Delta)

How much would the expected head gradient be near the delta, how fast would head decrease with depth.
Perhaps it would only go down a few meters for every layer

In [ ]:
distance = 5000
# Fine sand	2×10-7 to 2×10-4 m/s
# Silt, loess	1×10-9 to 2×10-5 m/s
# delta soils have some sand mixed in
delta_hk = scaling_factors.K_delta

ghbdelta_spd = pd.DataFrame(np.zeros(((nlay*nrow),5)))
ghbdelta_spd.columns = ['k','i','j','bhead','cond']

# get all of the j,k indices to reduce math done in the for loop
xz = np.zeros((nlay*nrow,2)).astype(int)
n=0
for i in np.arange(0,nrow):
    for k in np.arange(0, nlay):
        xz[n,0] = i
        xz[n,1] = k
        n+=1
cond = delta_hk*(top_botm[xz[:,1],:,0]-top_botm[xz[:,1]+1,:,0])*delr/distance
ghbdelta_spd.cond = cond
ghbdelta_spd.bhead = 0
ghbdelta_spd.k = xz[:,1]
ghbdelta_spd.j = 0
ghbdelta_spd.i = xz[:,0]

In [ ]:
# write to csv for use in UCODE file edits
ghbse_spd.to_csv(m.model_ws+'/input_data/ghbse_spd.csv',index=False)
ghbnw_spd.to_csv(m.model_ws+'/input_data/ghbnw_spd.csv',index=False)
ghbdelta_spd.to_csv(m.model_ws+'/input_data/ghbdelta_spd.csv',index=False)

In [ ]:
# lay, row, col for delta ghb
zxy = ghbdelta_spd.values[:,:3].astype(int)
# drop any delta ghb cells where cell bottom is below sea level
ghbdn_spd =  ghbdelta_spd.values[botm[zxy[:,0],zxy[:,1],zxy[:,2]]<0]
# join dataframes of 3 ghb boundaries together
ghb_spd = np.vstack((ghbdn_spd, ghbse_spd.values, ghbnw_spd.values))


In [ ]:

# GHB for east and west model boundaries
ghb = flopy.modflow.ModflowGhb(model=m, stress_period_data =  {0: ghb_spd},ipakcb=55)
# GHB for only Delta, west side of model
# ghb.stress_period_data =  {0: ghbdn_spd}

In [ ]:
ghb.check()

In [ ]:
# ghb.write_file()

## CHD Package Time variant head

In [ ]:
chd = flopy.modflow.ModflowChd(model=m,ipakcb=55)

In [ ]:
# historical observation data suggested groundwater levels near the foothills are on the range of 40 meters
# and have a steep gradient

In [ ]:
hob_dir = gwfm_dir+'/HOB_data'
# save cleaned data to box data for all data
all_obs = pd.read_csv(hob_dir+'/all_obs_grid_prepared.csv',index_col=0,parse_dates=True)
hill_obs = all_obs[all_obs.column>200].groupby('SITE_CODE').mean()
hill_obs
hill_grid = grid_p[grid_p.column>200][['row','column','geometry']]
hill_grid[['row','column']]-=1
hill_grid = hill_grid.join(hill_obs.set_index(['row','column']),on=['row','column'],how='inner')
hill_grid.plot('GWE',legend=True)

In [ ]:
# Determine which layer the specified head cell is in
# since the if statement only checks whether the first layer is greater than the streambed elevation, 
# otherwise it would be less than and zero (most should be in layer 0)

fig,ax=plt.subplots( figsize=(6,4))
ax.plot(dem_data[:,ncol-1])

chd_lay = np.zeros(nrow)

head = dem_data[:,ncol-1]
headmin = np.min(head)
ch_weight = 0.6
chd_vals = head*(1-ch_weight)+headmin*ch_weight
ax.plot(chd_vals,label='DEM ')

ch_weight = 0.8
chd_vals = np.full(dem_data[:,ncol-1].shape,40)
ax.plot(chd_vals,label='40 ')

chd_vals = head*(1-ch_weight)+40*ch_weight
ax.plot(chd_vals,label='DEM adj40 ')

plt.legend()
# kriged data are wildly inaccurate in the mountains so it is better to look at the two wells in that area individually
# kriged_mtn = np.vstack((kriged_spring[:,:,ncol-1], kriged_fall[:,:,ncol-1]))
# kriged_mtn_df = pd.DataFrame(np.rot90(kriged_mtn),columns=sy_ind)
# kriged_mtn_df.loc[:,'spring'].plot(ax=ax,linestyle='--')
# kriged_mtn_df.loc[:,'fall'].plot(ax=ax)

chd_lay2 = get_layer_from_elev(chd_vals, botm[:, :, -1], m.dis.nlay)

# for k in np.arange(0,nlay-1):
#     # pull out elevation of layer bottom
#     lay_elev = botm[k, :, ncol-1]
#     for i in np.arange(0,nrow):
#         # want to compare if streambed is lower than the layer bottom
#         # 1 will be subtracted from each z value to make sure it is lower than the model top in the upper reaches
#         if lay_elev[i] > chd_vals[i]:
#             chd_lay[i] = k



In [ ]:
# layer (int), row (int), column (int), shead (float), ehead (float) shead is the head at the
# start of the stress period, and ehead is the head at the end of the stress period
# nlay_ghb = 1

# constant head boundary for mountain front recharge
# assume that near the mountains the head should be at the surface becaues the aquifer is thin

# new specified head boundary will be linear at the uppermost column to reduce nonlinearity
# as the no flow cells will be removed and replaced with low hydraulic conductivity cells

# chd_spd = np.zeros((len(chd_locs[0]),5))
chd_spd = np.zeros((int(np.sum((nlay-chd_lay))),5))

# # head for mountain front recharge
shead = chd_vals
ehead = chd_vals
p=0
for i in np.arange(0,nrow):
    for k in np.arange(0,nlay-chd_lay[i]):
        chd_spd[p] = [int(chd_lay[i]+k), i, ncol-1, shead[i], ehead[i]]
        p+=1
print('Number of CHD cells for upland bound', p)

# p = 0
# # head for mountain front recharge
# shead = head
# ehead = head

# for i, j in zip(chd_locs[0], chd_locs[1]):
#     chd_spd[p] = [0, i, j, shead[p], ehead[p]]
#     p+=1
# print('Number of CHD cells for upland bound', p)


In [ ]:
# pretty big difference between starting heads and chd
# hd_strt[:,-1] - chd_vals

In [ ]:
# kriged_mtn = np.vstack((kriged_spring[:,:,ncol-1], kriged_fall[:,:,ncol-1]))
# kriged_mtn_df = pd.DataFrame(np.rot90(kriged_mtn),columns=sy_ind)

# fig,ax=plt.subplots( figsize=(6,4))
# # kriged_mtn_df.loc[:,'spring'].plot(ax=ax,linestyle='--')
# # kriged_mtn_df.loc[:,'fall'].plot(ax=ax)
# ax.plot(chd_vals)
# ax.plot(dem_data[:,ncol-1])

In [ ]:
chd.stress_period_data =  {0: chd_spd}

In [ ]:
chd.check()
# chd.write_file()

The code that originally imported the state soil map data was moved up above to be readily applicable to the LAK package and SFR package. gpd_mb and grid_uzf are created above

## RCH Package
1. Load in already interpolated ET and rainfall data
2. Determine where there is agricultural land to add irrigation multiplied by an efficiency factor to the rainfall array
3. Difference the rainfall and ETc arrays to dtermine how much water will recharge the aquifer

In [ ]:
ETc = np.zeros((nper_tr,nrow,ncol))
agETc = np.zeros((nper_tr,nrow,ncol))
rain = np.zeros((nper_tr,nrow,ncol))
non_dev_rain = np.zeros((nper_tr,nrow,ncol))
ag_rch = np.zeros((nper_tr,nrow,ncol))

ETc_count = 0
for y in np.arange(strt_date.year, end_date.year+1):
    # set start and end date for range for the year to be iterated over
    yr_strt = pd.to_datetime(str(y)+'-01-01')
    yr_end = pd.to_datetime(str(y)+'-12-31')
    # get the length of the date range needed for that year
    yearlen = len(pd.date_range(yr_strt, yr_end))
    if yr_strt < strt_date:
        yr_strt = strt_date
    if yr_end > end_date:
        yr_end = end_date
        
    # read in text file of all of the ETc data for each year in array format    
    ET_year = np.loadtxt(gwfm_dir+'/UZF_data/ETa_all_txt_arrays/ETa_array_'+str(y)+'.tsv', delimiter = '\t')
    rain_year = np.loadtxt(gwfm_dir+'/UZF_data/Rain_all_txt_arrays/Rain_array_'+str(y)+'.tsv', delimiter = '\t')
    all_ag_arr = np.loadtxt( gwfm_dir+'/WEL_data/simple_landuse_arrays/ag_land_'+str(y)+'.tsv',  delimiter='\t')
    dev_arr = np.loadtxt( gwfm_dir+'/WEL_data/simple_landuse_arrays/developed_land_'+str(y)+'.tsv',  delimiter='\t')
    non_dev_arr = ~dev_arr.astype(bool) # flip array to have non-developed land array for recharge estimates
    # estimate floodplain and Ag recharge
    natl_flood_y = np.loadtxt(gwfm_dir+'/WEL_data/simple_landuse_arrays/natl_flood_land_'+str(y)+'.tsv', delimiter='\t')
    ag_flood_arr = np.loadtxt(gwfm_dir+'/WEL_data/simple_landuse_arrays/ag_flood_land_'+str(y)+'.tsv', delimiter='\t')
    # aggregate floodplain array over years
    natl_flood_arr = np.where(natl_flood_y>0,1,0)
#     ag_flood_arr = np.where(ag_flood_y>0,1,0)
    
    # correct the shape of the text file from 2D to 3D
    revertETc = np.reshape(ET_year, (yearlen, nrow, ncol))
    revertrain = np.reshape(rain_year, (yearlen, nrow, ncol))
    # filter the 3D array based on the desired date range
    filtered_date_ETc = revertETc[yr_strt.dayofyear-1:yr_end.dayofyear,:,:]
    filtered_date_rain = revertrain[yr_strt.dayofyear-1:yr_end.dayofyear,:,:]
    # get the length of the date range needed for that year
    perlen = len(pd.date_range(yr_strt, yr_end))
    # add the data to the ETc array for the whole model time period
    ETc[ETc_count:ETc_count+perlen,:,:] = filtered_date_ETc
    agETc[ETc_count:ETc_count+perlen,:,:] = filtered_date_ETc*all_ag_arr
    rain[ETc_count:ETc_count+perlen,:,:] = filtered_date_rain
    non_dev_rain[ETc_count:ETc_count+perlen,:,:] = filtered_date_rain*non_dev_arr
    # extra recharge for alfalfa irrigation to offset lowered water levels
    ag_rch[ETc_count:ETc_count+perlen,:,:] = filtered_date_ETc*ag_flood_arr
    ETc_count += perlen

In [ ]:
# if len(ag_rch.shape) >2:
ag_rch_ss = ag_rch.mean(axis=0)

np.savetxt(model_ws+'/input_data/ag_rch_input.csv', np.reshape(ag_rch, ((nper_tr)*nrow,ncol)))

# np.savetxt(model_ws+'/input_data/ag_rch_input_ss.csv', ag_rch_ss)
# determine ag recharge by loss factor
ag_rch_scaled = ag_rch* scaling_factors.ag_rch_efficiency # assume 5% or 10% irrigation losses for now

In [ ]:
# copy over files from steady state input run
import shutil, os
files = ['base_rch_input_ss.csv', 'wel_input_ss.csv', 'ag_rch_input_ss.csv']
# np.savetxt(model_ws+'/input_data/base_rch_input_ss.csv', net_inf_ss)
ss_model_input = loadpth+'historical_SS/input_data/'

for f in files:
    f = ss_model_input + f
    shutil.copy(f, m.model_ws+'/input_data')

In [ ]:
np.savetxt(model_ws+'/input_data/natl_flood_arr.csv',natl_flood_arr)
inflow.to_csv(model_ws+'/input_data/inflow.csv', index=True)

# flood stage is 12 ft which corresponds to about 10,000 cfs so when flow is above 10,000 cfs
# 10000 cfs cutoff means it only floods in 2017, lets try 3,000 cfs floodplain connection limit
# 3,000 cfs gives 127 days which is 30 days per year
fp_days = (inflow[inflow.flow_cfs>scaling_factors.fp_active_flow].index - strt_date).days +1 # to account for steady state

fp_rch = np.zeros((nper_tr,nrow,ncol)) # floodplain recharge
# assume constant rate of 1 ft/day for days when flooding happens
fp_rch[fp_days,:,:] = np.ones((nrow,ncol))*natl_flood_arr*0.3048 
# reduce flooding by vka in layer 1
fp_rch = np.where(fp_rch > vka[0,:,:]*natl_flood_arr.astype(bool), vka[0,:,:]*natl_flood_arr.astype(bool), fp_rch)


# any locations more than 10,000 meters from Mokelumne Confluence are less likely to be floodplain inundation
fp_rch[:,:,scaling_factors.fp_column_extent.astype(int):]=0
      
fp_rch_ss = fp_rch.mean(axis=0)
# np.savetxt(model_ws+'/input_data/fp_rch_input_ss.csv', fp_rch_ss)


In [ ]:
# monthly recharge scaling
rch_factors = pd.DataFrame(scaling_factors[scaling_factors.index.str.contains(r'rch_[\d]', regex=True)])
rch_factors['month'] = rch_factors.index.str.extract(r'(\d+)').values.astype(int)
rch_factors = rch_factors.reset_index().set_index('month')

# right now monthly factors can't be applied directly with multiplication so need loop
# to apply scalars to appropriate indexes
rch_factors_long = rch_factors.loc[inflow.index.month.values,'StartValue']
rch_scale_arr = np.ones((nper_tr,nrow,ncol))

for n in rch_factors.index:
    rch_scale_arr[rch_factors_long.index==n,:,:] *= rch_factors.loc[n,'StartValue']

In [ ]:
# wherever there is ag land and not enough rain to meet the ETc then it will be met by applying 1.05*ETc
# to the .FINF array

# whereever the rain is less than ETc for ag land set the recharge equal to the etc times 0.05 for irrigation inefficiency
# could calibrate this irrigation efficiency later
# ag_rch = agETc

# final calculation for RCH package
# only consider rain on non-developed lands
net_inf = non_dev_rain  - ETc

# assuming there is irrigation to make up lack of rainfall
# net_inf should never be less than zero (ET shouldn't be used to take GW here, udpate to UZF for this)
net_inf = np.where(net_inf<0, 0, net_inf)
net_inf_out = np.reshape(net_inf, ((nper_tr)*nrow,ncol))
np.savetxt(model_ws+'/input_data/base_rch_input.csv', net_inf_out)

# scale net_inf by monthly factors
net_inf *= rch_scale_arr

# if the net_inf is greater than the vertical hydraulic conductivity then recharge should be rejected in RCH
# this is automated in UZF as well
# print('%.2f %% of cells in all time should have rejected recharge' %((net_inf >vka[0,:,:]).sum()*100/(nper_data*nrow*ncol)))
net_inf_all = np.copy(net_inf)
net_inf = np.where(net_inf >vka[0,:,:], vka[0,:,:],net_inf)
# areas with steep slopes such as in the foothills should also see a reduced recharge (add parameter)
net_inf[:,deep_geology[0,:,:]] *= 0.75

# take average for steady state
net_inf_ss = net_inf.mean(axis=(0))
# save as input_file
# np.savetxt(model_ws+'/input_data/base_rch_input_ss.csv', net_inf_ss)

# don't reduce these terms by the limiting infiltration rate assuming
# that they are not running off, but rather ponded due to berms for alfalfa or landscape for flooding
# add recharge expected due to alfalfa irrigation
#net_inf += ag_rch_scaled

# add recharge expected due to floodplain inundation
# net_inf += fp_rch

In [ ]:
rch_chk = pd.DataFrame(net_inf.mean(axis=(1,2)), columns = ['rch'])
rch_chk['rch_red'] = np.where(net_inf_all >vka[0,:,:], vka[0,:,:],net_inf_all).mean(axis=(1,2))
rch_chk['dt'] = pd.date_range(strt_date, end_date)
rch_chk = rch_chk.set_index('dt')
rch_chk.resample('M').sum().plot()
rch_chk.resample('A').sum().plot()

In [ ]:
# have transient recharge start after the 1st spd

rech_spd = { (j): net_inf[j-1,:,:] for j in np.arange(time_tr0,nper)}

In [ ]:
# set average recharge for whole period as steady state
# consider reducing by some factor to avoid putting too much flow in steady state, 
# maximum recharge for all time is 10 cm, mean gives only 2mm so it should be fine
if no_ss == False:
    rech_spd[0] = net_inf_ss*scaling_factors.UZF_s  + fp_rch_ss# + ag_rch_ss*scaling_factors.ag_rch_efficiency  #

In [ ]:
# nrchop = 3, to highest active cell
rch =flopy.modflow.ModflowRch(model = m, nrchop=3, rech = rech_spd, ipakcb=55)

In [ ]:
# rch.write_file()

## Well Package


In [ ]:
wells_grid = pd.read_csv(gwfm_dir+'/WEL_data/wells_grid.csv')
wells_grid = gpd.GeoDataFrame(wells_grid, geometry = gpd.points_from_xy(wells_grid.easting, wells_grid.northing),
                              crs='epsg:32610')

In [ ]:
ag_well_depth_arr = np.loadtxt(gwfm_dir+'/WEL_data/ag_well_depth_arr.tsv', delimiter='\t')

### Calculate layers for Ag wells then public/domestic
Dependent on m.dis.botm

In [ ]:
ag_screen_botm = np.where(dem_data-ag_well_depth_arr<topbotm)
ag_screen_botm = np.rot90(np.vstack(ag_screen_botm))
ag_screen_botm = pd.DataFrame(ag_screen_botm, columns=['layer','row','column'])
ag_max_lay = ag_screen_botm.groupby(['row','column']).max()
# any wells below most bottom go in bottom layer
ag_max_lay.layer[ag_max_lay.layer == nlay] = nlay-1

# assume 10% of well is screened? Pauloo? tprogs lay thickness is 4m, so 12ft, not quite enough for typical well?
# if we go two layers we have 8 m which is near the average expected well screen
ag_screen_top = np.where((dem_data-ag_well_depth_arr*0.9)<topbotm)
ag_screen_top = np.rot90(np.vstack(ag_screen_top))
ag_screen_top = pd.DataFrame(ag_screen_top, columns=['layer','row','column'])
ag_min_lay = ag_screen_top.groupby(['row','column']).max()
ag_min_lay.layer[ag_min_lay.layer == nlay] = nlay-1


In [ ]:
wel_bot_elev = dem_data[wells_grid.row.values, wells_grid.column.values] - wells_grid.depth_m

wells_grid.layer = get_layer_from_elev(wel_bot_elev, botm[:,wells_grid.row.values, wells_grid.column.values], m.dis.nlay)

In [ ]:
print('Cells with wells: ', wells_grid.dissolve(by='node',aggfunc='first').shape[0], 'total wells: ', wells_grid.shape[0])
print('Wells with TRS accuracy: ', (wells_grid.MethodofDeterminationLL == 'Derived from TRS').sum())

wells_grid_notrs = wells_grid.loc[wells_grid.MethodofDeterminationLL != 'Derived from TRS']

In [ ]:
num_ag = (wells_grid.Simple_type == 'irrigation').sum()
# need to correct ag_land_arr, so where there is pasture/grass then we look to verify if there is an ag well
# already filtering by land type above
ET_ag = np.copy(agETc)
# remove ET_ag that causes pumping depression in foothills that isn't backed by head data
# only use layer 1 to limit impact and this relates to where the issue is
ET_ag = ET_ag*(~deep_geology[0,:,:])
# summarize to monthly for estimating monthly use
ET_ag_monthly_sum = ET_ag.sum(axis=(1,2))
# calculate ag well flux by average ET
aggregated_irrig_flux = ET_ag_monthly_sum/num_ag


In [ ]:
# assume each public supply well serves 5-10,000 people each needing 50 gpd, then need to convert to ft^3
public_flux = (5000*50/7.48)*(0.3048**3)
# public_flux = 1500*(1/7.48)*(0.3048**3)*60*24

# averge pumping rate of domestic wells is 10-100 gpm, typically on the lower end and 
# should end up being around 50 gal/person a day and average 3 people is 150 gal/day
# 10 gpm * (1 ft^3/ 7.48 gal) (ft^3/m^3)*60 min hour* 3 hours in a day * 30 days

# dom_flux = 30*(1/7.48)*(0.3048**3)*60*24 # too large
dom_flux = 100*3*(1/7.48)*(0.3048**3)

print('Irrig flux:', '%.3e' % (aggregated_irrig_flux.mean()*200*200), 'Public flux:', '%.3e' %public_flux,'Domestic flux:', '%.3e' %dom_flux)

In [ ]:
# define pumping rate based on well use, average pumping rate in m^3/day
# wells_grid.loc[wells_grid.Simple_type == 'irrigation', 'flux'] = -irrig_flux
wells_grid.loc[wells_grid.Simple_type == 'domestic', 'flux'] = -dom_flux
wells_grid.loc[wells_grid.Simple_type == 'public', 'flux'] = -public_flux

In [ ]:
# wells_grid_notrs['count_per_cell'] = 1
# # all_wells_grid['MFCell'] = all_wells_grid.Township+all_wells_grid.Range+all_wells_grid.k.astype(str)
# wells_grid_sum = wells_grid_notrs.dissolve(by = 'node', aggfunc='sum')
# wells_grid_sum.plot('count_per_cell',legend=True)
# print(wells_grid_sum.count_per_cell.median(), wells_grid_sum.count_per_cell.mean(), wells_grid_sum.count_per_cell.max())


In [ ]:
# save domestic, public wells for their pumping doesn't depend on ET
wells_grid_no_ag = wells_grid.loc[wells_grid.Simple_type != 'irrigation']
# remove wells older than 40 years for ag-residential (no impact)
# 500 wells are removed with 30 year age, 1500 remain. Looks to be even removal throughout
wells_grid_no_ag = wells_grid_no_ag[wells_grid_no_ag.well_age_days < 30*365]


# filter out data for wel package
spd_noag = wells_grid_no_ag.loc[:,['layer','row','column', 'flux']].values


#### Using ETc in all cells for pumping

In [ ]:
# ag_max_lay = ag_min_lay+1

# iterate over all row, col and get layers for each well based on "screen" 
ag_well_lay = np.zeros((1,3))
for i,j in zip(ag_min_lay.reset_index().row,ag_min_lay.reset_index().column):
    lays = np.arange(ag_min_lay.layer.loc[i,j],ag_max_lay.layer.loc[i,j]+1)
    ijk = np.rot90(np.vstack((np.tile(i,len(lays)), np.tile(j,len(lays)),lays)))
    ag_well_lay = np.vstack((ag_well_lay,ijk))
# delete filler first row
ag_well_lay = ag_well_lay[1:]
ag_well_lay = pd.DataFrame(ag_well_lay.astype(int), columns=['row','column','layer'])

In [ ]:
num_ag_layers = (ag_max_lay - ag_min_lay+1).reset_index()
# divide ET_ag by the number of layers it will go into
ET_ag_layered = np.copy(ET_ag)
ET_ag_layered[:,num_ag_layers.row,num_ag_layers.column] /= num_ag_layers.layer.values
# adjustments to allow connection with rows,cols with pumping
row_col = ag_well_lay.loc[:,['row','column']].rename({'row':'rowi','column':'colj'},axis=1)
ag_well_lay = ag_well_lay.set_index(['row','column'])
ag_well_lay['rowi'] = row_col.rowi.values
ag_well_lay['colj'] = row_col.colj.values

In [ ]:
# plt.imshow(ET_ag_layered.mean(axis=0))
# plt.colorbar()

In [ ]:
m3_AF = 1/(0.3048**3)/43560

In [ ]:
# check ag pumping 
# pump_check = pd.DataFrame(-ET_ag.sum(axis=(1,2))*200*200,columns=['Ag_pump_m3_day'])
# pump_check['Date'] = pd.date_range(strt_date, end_date)
# pump_check['domestic_m3_day'] = wells_grid_no_ag[wells_grid_no_ag.Simple_type =='domestic'].flux.sum()
# pump_check['public_m3_day'] = wells_grid_no_ag[wells_grid_no_ag.Simple_type =='public'].flux.sum()
# pump_check = pump_check.set_index('Date')
# pump_check.resample('AS-Oct').sum()*m3_AF

In [ ]:
# layer for ETc ag well pumping
# ETc_lay = 1
# create empty dictionary to fill with stress period data
wel_ETc_dict = {}
# end date is not included as a stress period, starting at 1st TR spd (2)
for t in np.arange(time_tr0,nper):
    wel_i, wel_j = np.where(ET_ag[t-time_tr0,:,:]>0)
    new_xyz = ag_well_lay.loc[list(zip(wel_i,wel_j))] 
#     wel_ETc = -ET_ag[t-1,wel_i,wel_j]*delr*delr
# use new row,cols because there are more layers to use
    wel_ETc = -ET_ag_layered[t-time_tr0,new_xyz.rowi,new_xyz.colj]*delr*delr
    # ['layer','row','column', 'flux'] are necessary for WEL package
    spd_ag = np.stack((new_xyz.layer, new_xyz.rowi, new_xyz.colj,wel_ETc),axis=1)
    # correct by dropping any rows or cols without pumping as some may be added
    spd_ag = spd_ag[spd_ag[:,-1]!=0,:]
    # join pumping from ag with point pumping from domstic/supply wells that are constant
    spd_all = np.vstack((spd_ag,spd_noag)) 
    wel_ETc_dict[t] = spd_all

In [ ]:
# still a few wells in foothills that might be causing the drawdown, but more localized now
# test = np.zeros((nlay,nrow,ncol))
# test[spd_noag[:,0].astype(int),spd_noag[:,1].astype(int),spd_noag[:,2].astype(int)]=spd_noag[:,3]
# plt.imshow(test.sum(axis=0))

In [ ]:
if no_ss ==False:
    ET_ag_ss = ET_ag_layered.mean(axis=(0))*scaling_factors.WEL_s
    wel_i, wel_j = np.where(ET_ag_ss[:,:]>0)
    new_xyz = ag_well_lay.loc[list(zip(wel_i,wel_j))] 

    wel_ETc = -ET_ag_ss[new_xyz.rowi,new_xyz.colj]*delr*delr
    # ['layer','row','column', 'flux'] are necessary for WEL package
    # add average ag well pumping as steady state period
    spd_ag = np.stack((new_xyz.layer, new_xyz.rowi, new_xyz.colj,wel_ETc),axis=1)
    # correct by dropping any rows or cols without pumping as some may be added
    spd_ag = spd_ag[spd_ag[:,-1]!=0,:]
    # join pumping from ag with point pumping from domstic/supply wells that are constant
    spd_all = np.vstack((spd_ag,spd_noag)) 
    wel_ETc_dict[0] = spd_all

In [ ]:
# t=100
# plt.imshow(ET_ag[t,:,:]*200*200)
# plt.colorbar()

In [ ]:
# Create well flopy object
wel = flopy.modflow.ModflowWel(m, stress_period_data=wel_ETc_dict,ipakcb=55)

In [ ]:
# wel.write_file()

## HOB Package

In [ ]:
hob_dir = gwfm_dir+'/HOB_data'

In [ ]:
# save cleaned data to box data for all data
all_obs = pd.read_csv(hob_dir+'/all_obs_grid_prepared.csv',index_col=0,parse_dates=True)

# all_obs = all_obs.set_index('dt')
all_obs = all_obs.loc[(all_obs.index>strt_date)&(all_obs.index<end_date)]

# # get spd corresponding to dates
all_obs['spd'] = (all_obs.index-strt_date).days.values


In [ ]:
# any well with less than 2 measurements per year the period should not be used
# difference between 2 and 4 measurements dropped is only 3 more wells so conservative is better
hob_min = int(2*nper/365)-2
# wells to be used in calibration
hob_use = all_obs.SITE_CODE.unique()[(all_obs.groupby('SITE_CODE').count()>hob_min)["GWE"].values]
print('Number of wells droppped', all_obs[~all_obs.SITE_CODE.isin(hob_use)].SITE_CODE.unique().shape[0])
print('Number of wells kept',all_obs[all_obs.SITE_CODE.isin(hob_use)].SITE_CODE.unique().shape[0])

all_obs = all_obs[all_obs.SITE_CODE.isin(hob_use)]

In [ ]:
# fill wells with missing depth data based on depth interpolation map
all_obs.loc[all_obs.WELL_DEPTH.isna(), 'WELL_DEPTH'] = ag_well_depth_arr[all_obs.row, all_obs.column][all_obs.WELL_DEPTH.isna()]

obs_bot_elev = dem_data[all_obs.row, all_obs.column] - all_obs.WELL_DEPTH

all_obs['layer'] = get_layer_from_elev(obs_bot_elev, botm[:,all_obs.row, all_obs.column], m.dis.nlay)

In [ ]:
# save cleaned data to input data for post-processing
all_obs.to_csv(model_ws+'/input_data/all_obs_grid_prepared.csv')

In [ ]:
dwr_site_codes = all_obs.SITE_CODE.unique()

# create a new hob object
obs_data = []
dwr_spd_wse = all_obs.loc[:,['SITE_CODE','spd','GWE']]
for i in np.arange(0,len(dwr_site_codes)): # for each well location
    # time_series_data needs stress period in col 0 and WSE in col 1
    # the row and column are already one based as they come from grid_p
    # need to return and adjust layer later, for now layer 2 should be good so that it doesn't go dry
    # get stress period data and water surface elevation for well
    dwr_site = all_obs.set_index('SITE_CODE').loc[dwr_site_codes[i]]
    if dwr_site.ndim >1:
        row = dwr_site.row.values[0]
        col = dwr_site.column.values[0]
        layer = dwr_site.layer.values[0]
        names = dwr_site.obs_nam.tolist()
        obsname = 'N'+str(dwr_site.node.values[0])
    else:
        row = dwr_site.row
        col = dwr_site.column
        layer = dwr_site.layer
        names = dwr_site.obs_nam
        obsname = 'N'+str(dwr_site.node)
        
    tsd = dwr_spd_wse.set_index('SITE_CODE').loc[dwr_site_codes[i]].values
    # need to minus 1 for grid_p which is 1 based
    temp = flopy.modflow.HeadObservation(m, layer=layer, row=row, 
                                                  column=col,
                                                  time_series_data=tsd,
                                                  obsname=obsname, names = names)
    # correct time offset from stress period to be 0
    temp.time_series_data['toffset'] = 0
    obs_data.append(temp)


In [ ]:
hob = flopy.modflow.ModflowHob(m, iuhobsv=50, hobdry=-9999., obs_data=obs_data, unitnumber = 39,
                              hobname = 'MF.hob.out')

In [ ]:
# hob.write_file()

# Stream Gage outflow

In [ ]:
sensors = pd.read_csv(gwfm_dir+'/Mapping/allsensor_latlong.csv')
sensors = gpd.GeoDataFrame(sensors,geometry=gpd.points_from_xy(sensors.Longitude, sensors.Latitude))
sensors.crs = 'epsg:4326'
sensors = sensors.to_crs('epsg:32610')
mcc_grid = gpd.sjoin(sensors[sensors.Site_id=='MCC'],xs_sfr)
mcc_grid.columns

In [ ]:
# numgage is total number of gages
# gage_data (list, or array), includes 2 to 3 entries (LAKE -UNIT (OUTTYPE)) for each LAK entry
#  4 entries (GAGESEG, GAGERCH, UNIT, OUTTYPE) for each SFR package entry

mcc_gage_data = [[mcc_grid.iseg, mcc_grid.reach_new, 37, 1]]
gage_file = 'MF.gage'
mcc_file_out = 'MF_mcc.go' # not recognized still
gag = flopy.modflow.ModflowGage(model=m,numgage= 1,gage_data=mcc_gage_data,
                                filenames =[gage_file,mcc_file_out])


In [ ]:
# gag.write_file()
# m.write_name_file()

## Output Control

In [ ]:
# Output control
# default unit number for heads is 51, cell by cell is 53 and drawdown is 52
# (0,0) is (stress period, time step)

# get the first of each month to print the budget
month_intervals = (pd.date_range(strt_date,end_date, freq="MS")-strt_date).days
spd = {}
# output file for parallel runs when head/cbc is not needed
for j in month_intervals:
    spd[j,0] = ['print budget']
oc = flopy.modflow.ModflowOc(model = m, stress_period_data = spd, compact = True, filenames='MF_parallel.oc')
oc.write_file()


# For later model runs when all the data is needed to be saved
spd = {}
spd = { (j,0): ['save head', 'save budget'] for j in np.arange(0,nper,1)}

for j in month_intervals:
    spd[j,0] = ['save head', 'save budget','print budget']
    
oc = flopy.modflow.ModflowOc(model = m, stress_period_data = spd, compact = True)

## Newton Solver

In [ ]:
nwt = flopy.modflow.ModflowNwt(model = m, headtol=1E-4, fluxtol=500, maxiterout=200, thickfact=1e-05, 
                               linmeth=1, iprnwt=1, ibotav=0, options='Specified')
nwt_dict = nwt.__dict__

# load in parameters used by margaret shanafield for DFW
nwt_ex = pd.read_csv(gwfm_dir+'/Solvers/nwt_solver_input_from_dfw.csv', comment='#')
nwt_ex['nwt_vars'] = nwt_ex.NWT_setting.str.lower()
nwt_ex = nwt_ex.set_index('nwt_vars')
nwt_ex = nwt_ex.dropna(axis=1, how='all')
# nwt_ex.select_dtypes([float, int])

for v in nwt_ex.index.values:
    nwt_dict[v] = nwt_ex.loc[v,'Second'].astype(nwt_ex.loc[v,'nwt_dtype'])
    
# correct fluxtol for model units of m3/day instead of m3/second
nwt_dict['fluxtol'] = 500 
    # update NWT sovler parameters
nwt.__dict__ = nwt_dict

nwt.write_file()

In [ ]:
# pcg = flopy.modflow.ModflowPcg(model = m)
# nwt = flopy.modflow.ModflowNwt(model= m)
# thickfact: portion of cell thickness used for smoothly adjusting storage and conductance coefficients to zero (default is 1e-5)
# linmeth (linear method): 1 for GMRES and 2 for XMD (1 is default)
# iprnwt: flag whether additional info about solver convergence will be printed to the main listing file (default is 0)
# ibotav: flag whether corretion will be made to gw head relative to cell-bottom if surrounded by dry cells.
# 1 = corrections and  0 = no correction (default is 0)
# options: specify comlexity of solver. SIMPLE : default solver for linear models, MODERATE for moderately nonlinear models,
# COMPLEX for highly nonlinear models (default is COMPLEX)
# Continue: if model fails to converge during a time step it will continue to solve the next time step (default is False) 
# epsrn (XMD) is the drop tolerance for preconditioning (default is 1E-4)
# hclosexmd (XMD) head closure criteria for inner (linear) iterations (default 1e-4)



In [ ]:
# max_outer = 200
# max_inner = 100
# hclose = 1e-02
# fluxtol = 500
# rclose=1e-01


# iter_mo max outer iters, ter_mi = max inner iters, close_r residual criterion for stopping iteration
# close_h is alternate criterion for nonlinear problem, and is head closure which should be smaller than residual closer
# ipunit =0 means no info on solver, ipunit=1 means output about solver issues is written
# if iter_mo >1 then closer_r is used not close_h and closer_r is compared to 
# the square root of the inner product of the residuals (the residual norm)
# adamp =0 is std damping, adamp=1 is adaptive damping that further decreases or increases damping based on picard
# iteration sucess
#adamp is 0.7 to resolve issues with heads oscillating near solution +1 m
# damp_lb = lower bound, rate_d is rate of increase of damping based picard iteration success

# from SVIHM, if you rewrite the solver parameters in NWT it can change between stress periods
# solver = flopy.modflow.ModflowPcgn(m, iter_mo = max_outer, iter_mi=max_inner, close_r=rclose, close_h=hclose, ipunit=28) 
#                                 relax = 0.99, ifill=1)
#                                adamp=1, damp=0.7, damp_lb=0.1, rate_d=0.01)



In [ ]:
# solver.write_file()

In [ ]:
m.get_package_list()
# m.remove_package('DATA')
# m.remove_package('LAK')
# m.remove_package('WEL')
# m.remove_package('RCH')
# m.remove_package('NWT')

In [ ]:

m.check()
# lak.check()
# upw.check()

## Write the input files

In [ ]:
# save modflow workspace file to WRDAPP sub folder to improve runtime calculations
# loadpth = 'F:/WRDAPP/GWFlowModel/Cosumnes/levee_setback/streamflow/'
# # load model with only DIS to reduce load time
# # the model will run off of the .nam file connection so flopy doesn't need them
# all_model_ws = loadpth + '/historical_streamflow'


# load_only = ['CHD', 'SFR','PCGN','OC','BAS6','DIS'] # avoid loading packages that are entirely re-written
# # in the case this runs in the basefolder then the current working directory
# # is the same as the all working directory - useful now with weird naming between realizations
# # if basename(model_ws).__contains__('historical'):
# #     all_model_ws = model_ws
# m2 = flopy.modflow.Modflow.load('MF.nam', model_ws=all_model_ws, 
#                                 exe_name='mf-owhm.exe', version='mfnwt')

In [ ]:
# Writing the MODFLOW data files
m.write_input()


# Run the model

In [ ]:
# success, buff = m.run_model()

# Model notes
Maybe due to model complexity I should simplify model to 3 layers for inital calibration and then run individual sensitivity analysis rather than calibration looking at gwe data for SOSWR then streamflow data.
I should check at least a couple other realizations to check fit
Need to add streamflow calibration data for McConnell.

Big question: is the drawdown in the basin primarily due to internal groundwater pumping from agriculture and domestic wells or from external outflow. I may need to increase the outflow to the GHB to bring down water levels in the basin. Jan used simulated gw head 1000 m from the model edge and the arithmetic mean of the regional flow model hk

I should also return to look at whether the nwt solver will work and then look at unconfined simulation. Jan only simulated the top layer as unconfined to represent the water table.

Sy is only read for converitble layers (LAYTYP != 0)
Ss is specific storage unless STORAGECOEFFICIENT is used then it is the confined storage coefficient
Sand/Gravel hk> ~100

Running one parallel run took about 44 minutes but no major issues so i'm unsure why the parallel calibration failed
First forward run took about 47 minutes as well still using PCGN (hits mxiter 33 times mbe = 0.00, mxiter=200), try switching to NWT and playing with run time
NWT solver takes 1 hr 46 minutes and hits mxiter 11 times (mxiter=500), mbe = 0.00% with COMPLEX setting.
NWT solver with MODERATE takes 1 hr 39 minutes mxiter 11 times (mxiter=300), mb = 0.02
Using base NWT settings it is slower than PCGN

# UCODE Input

In [ ]:
# pgroup = pd.read_csv(model_ws+'/MF_ucode.pgroup', delimiter=r'\s+', index_col='GroupName',
#                      skipfooter=1, skiprows=2, engine='python')
# pgroup.loc['GHB','Adjustable']
# pgroup.index
# # if pgroup.loc['UZF','Adjustable'] =='yes':
# #     print('True')

## Format parameter data (pdata) file

In [ ]:
def get_magnitude(x):
    return(10.0**(np.log10(x).astype(int)))


# melt parameter data and rename columns to fit UCODE format for .pdata
pdata = params.rename(columns={'K_m_d':'Kx'})[['Kx','vani','Ss','Sy']]
pdata = pdata.melt(ignore_index=False)
pdata['ParamName'] = pdata.variable + '_' + pdata.index.astype(str)
pdata = pdata.rename(columns={'variable':'GroupName','value':'StartValue'}).reset_index(drop=True)
# join scaling factors to hydraulic parameters
pdata = pdata.append(scaling_factors_all)

# default values for pdata input
pdata['LowerValue'] = 1E-38
pdata['UpperValue'] = 1E38

# local adjustment based on typical parameter scaling (start value scaled by a range)
# need to find a better rounding function
grps = pdata.GroupName.isin(['Kx','Ss','vani','GHB'])
pdata.loc[grps,'LowerValue'] = get_magnitude(pdata.loc[grps,'StartValue']) *1E-3
pdata.loc[grps,'UpperValue'] = get_magnitude(pdata.loc[grps,'StartValue']) *1E3
grps = pdata.GroupName.isin(['Sy'])
pdata.loc[grps,'LowerValue'] = get_magnitude(pdata.loc[grps,'StartValue']) *1E-2
pdata.loc[grps,'UpperValue'] = 1
grps = pdata.ParamName.str.contains('rch_')
pdata.loc[grps,'LowerValue'] = get_magnitude(pdata.loc[grps,'StartValue']) *1E-3
pdata.loc[grps,'UpperValue'] = 2

# assume constraints align with expected range
pdata['Constrain'] = 'No'
pdata['LowerConstraint'] = pdata.LowerValue
pdata['UpperConstraint'] = pdata.UpperValue



In [ ]:
# pdata.ParamName.str.contains('rch_')
# pdata.ParamName.isin([r'rch_.'])


## JTF files

In [ ]:
# Write out jtf file
p_out = params.drop(columns=['K_m_d'])
p_out.K_m_s = '@'+('Kx_'+p_out.index.astype(str)).str.ljust(20)+'@'
# p_out.vani = '@'+('vani_'+p_out.index.astype(str)).str.ljust(20)+'@'
p_out.Sy = '@'+('Sy_'+p_out.index.astype(str)).str.ljust(20)+'@'
p_out.Ss = '@'+('Ss_'+p_out.index.astype(str)).str.ljust(20)+'@'

with open(model_ws+'/ZonePropertiesInitial.csv.jtf', 'w',newline='') as f:
    f.write('jtf @\n')
    p_out.to_csv(f,index=True, mode="a")
    
scaling_jtf = scaling_factors_all.copy()
# Write out jtf file
scaling_jtf.StartValue = '@'+scaling_jtf.ParamName.str.ljust(20)+'@'

with open(model_ws+'/GHB_UZF_WEL_scaling.csv.jtf', 'w',newline='') as f:
    f.write('jtf @\n')
    scaling_jtf.to_csv(f,index=False, mode="a")

## Observation data

In [ ]:
ucode_fxn_dir = doc_dir+'/GitHub/CosumnesRiverRecharge/ucode_utilities'
if ucode_fxn_dir not in sys.path:
    sys.path.append(ucode_fxn_dir)
# sys.path
import ucode_input

from importlib import reload
# importlib.reload
reload(ucode_input)

In [ ]:
hobout = pd.read_csv(m.model_ws+'/MF.hob.out',delimiter = r'\s+')

# here std deviation represents the actual value one expects
# for a well the accuracy is 0.01 ft at best based on measuring tape scale
all_obs['Statistic'] = 0.01
all_obs['StatFlag'] = 'SD'
# locations with significant difference between RPE GSE and the DEM should have additional uncertainty included
all_obs['Statistic'] += np.round(np.abs(all_obs.dem_wlm_gse),4)

hobout_in = hobout.join(all_obs.set_index('obs_nam')[['Statistic','StatFlag']],on=['OBSERVATION NAME'])
# temporary fix for misjoin for single observation HOB nodes
hobout_in.loc[hobout_in.Statistic.isna(),'Statistic'] = 0.01 
hobout_in['StatFlag'] = 'SD'

ucode_input.write_hob_jif_dat(model_ws, hobout_in, statflag=True)

In [ ]:
sfr_dir = gwfm_dir+'/SFR_data/'

In [ ]:
# data for obs table
mcc_d = pd.read_csv(sfr_dir+'MCC_flow_obs_all.csv', index_col='DATE TIME', parse_dates=True)

mcc_d = mcc_d[(mcc_d.index>=strt_date)&(mcc_d.index<=end_date)]
# ObsName ObsValue Statistic StatFlag GroupName
mcc_d['ObsName'] = ('mcc_'+pd.Series(np.arange(0,len(mcc_d)).astype(str)).str.zfill(5)).values
# make sure units are flow in m^3/day
mcc_d = mcc_d.rename(columns={'flow_cmd':'ObsValue'})

cols_out = ['ObsName','ObsValue','Statistic','StatFlag','GroupName']

header = 'BEGIN Observation_Data Table\n'+\
    'NROW= '+str(len(mcc_d))+' NCOL= 5 COLUMNLABELS\n'+\
    ' '.join(cols_out)

footer = 'End Observation_Data'
# get array of just strings
flow_arr = mcc_d[cols_out].values
# pull out observed value and name of obs
np.savetxt(model_ws+'/flow_obs_table.dat', flow_arr,
           fmt='%s', header = header, footer = footer, comments = '' )

# for gage file JIFs need to specify which flows are used by specify the observation name
# for the correct row (time) and filling the rest with a dummy variable (Cab used dum)

In [ ]:
gagenam = model_ws+'/MF.gage1.go'
gage = pd.read_csv(gagenam,skiprows=1, delimiter = r'\s+', engine='python')
# clean issue with column name read in
cols = gage.columns[1:]
gage = gage.dropna(axis=1)
gage.columns = cols
# set datetime for joining with flow obs data
gage['dt'] = strt_date + (gage.Time-1).astype('timedelta64[D]')
gage = gage.set_index('dt').resample('D').mean()
gage_jif = gage[['Time','Flow']].join(mcc_d)
# if I leave Nan values then ucode gets upset, Cab used the filler dum which I think Ucode identifies
gage_jif.loc[gage_jif.ObsName.isna(),'ObsName'] = 'dum'

In [ ]:
# def write_flw_jif(model_ws, gagout):
#     # skip 2 rows, use 3rd column values for 1345 values for std MF gage out file
header = 'jif @\nStandardFile 2 3 '+str(len(gage_jif))
# header = 'jif @\n'+'StandardFile  1  1  '+str(len(obsoutnames))
# obsoutnames.to_file(m.model_ws+'/MF.hob.out.jif', delimiter = '\s+', index = )
np.savetxt(model_ws+'/MF.gage1.go.jif', gage_jif.ObsName.values,
           fmt='%s', delimiter = r'\s+',header = header, comments = '')

## Parallel ucode

In [ ]:
n_nodes = ucode_input.get_n_nodes(13)

In [ ]:
# 2400 seconds is about 40 minutes which is avg run time
# may need to extend upward to 3 hours (10800) for slow runs
ucode_input.write_parallel(model_ws, n_nodes,2400) 

In [ ]:
# copy mf files except cbc and hds
mf_files = pd.Series(glob.glob(m.model_ws+'/MF.*'))
mf_files = mf_files[~mf_files.str.contains('cbc|hds').values].tolist()
jtfs = glob.glob(m.model_ws+'/*.jtf')
run = glob.glob(m.model_ws+'/*py*')

files = mf_files+jtfs+run
files = glob.glob(m.model_ws+'/*.jtf')

In [ ]:
# when dealing with larger data sets, it may be worthwhile using parallel subprocess
import shutil, os


for n in np.arange(0, n_nodes).astype(str):
    folder = '/r'+ n.zfill(3)+'/'
    os.makedirs(m.model_ws+folder,exist_ok=True)
    for f in files:
        shutil.copy(f, m.model_ws+folder)

In [ ]:
# replace oc file with simplified version that only prints the budget monthly
f = glob.glob(m.model_ws+'/MF_parallel.oc')[0]

for n in np.arange(0, n_nodes).astype(str):
    folder = '/r'+ n.zfill(3)+'/'
    os.makedirs(m.model_ws+folder,exist_ok=True)
    shutil.copy(f, m.model_ws+folder+'/MF.oc')

In [ ]:
import shutil, os

# write out just the updated python write file
write_file = glob.glob(model_ws+'/*.py')
for n in np.arange(0,n_nodes).astype(str):
    folder = '/r'+ n.zfill(3)
    shutil.copy(write_file[0], model_ws+folder)